# GQ identification scheme

See also https://link.springer.com/content/pdf/10.1007/3-540-45708-9_11.pdf

The Guillou-Quisquater (GQ) identification scheme was defined in 1988 [1](https://link.springer.com/content/pdf/10.1007/0-387-34799-2_16.pdf) and supports a zero knowledge proof method. The prover (Peggy) has a proving public key of $(N,e,X)$ where $N$ is the modulus, $e$ is the exponent, and $X=x^e \mod N)$. $x$ is the secret value that the prover (Peggy) must prove $(x \in \mathbb{Z}^*_N)$. After Peggy generates his public proving key, she will then be challenged by Victor to produce the correct result. In the following we define a secret value $(x)$, an exponent $(e)$ and a modulus value $(N)$

Note: We are using a small prime number and a small value of $x$ here in order to illustrate the process. In real life the prime number would have at least 2,048 bits, and the random value would be based on the 256-bit hashed value of the secret.

Note: $x$ and also $y$ must be relative prime to $N$, and where they must not share any factors. We must thus check that $x$ and $y$ do not have a common factor with $N$. We normally do this be determining the greatest common divisor (GCD), and if greater than 1, we will fail the value.

$
X \equiv x^e \mod N \\
Y \equiv y^e \mod N \\
Z \equiv yx^c \mod N \\
YX^c \equiv X^e \mod N \\
YX^c = y^e(x^e)^c = y^ex^{ec} \\
Z^e = (yx^c)^e = y^ex^{ec}
$

In [1]:
from Crypto.Util import number

In [14]:
# Peggy generates 
N = number.getPrime(512) # public
x = number.getRandomNBitInteger(16) # known secret
e = number.getRandomNBitInteger(4) # public
y = number.getRandomNBitInteger(16) # random, private

# Peggy computes 
X = pow(x,e,N) # Part of public key
Y = pow(y,e,N) # Sent to Victor

assert (number.GCD(X, N) == 1) == (number.GCD(Y, N) == 1), "X and Y cannot share factors with N"

# Victor 
c = number.getRandomNBitInteger(32) # random challenge sent to Peggy

# Peggy
Z = pow(y * x, c, N) # Sent to Victor

# Victor
val1 = pow(X*Y, c, N)
val2 = pow(Z,e,N)

print("Peggy knows value of x") if val1 == val2 else print("Peggy did not prove knowledge of x")

Peggy knows value of x


# Contact tracing

* [Epione protocol](https://arxiv.org/pdf/2004.13293.pdf) is based on a two party PSI cardinality proof. 
* Alice and Bob both have sets and after the protocol learn only the cardinality of their joint set intersection

The protocol works as follows:

* Alice and Bob agree on the curve $G$
* Bob has values $a_i$ and matches these with the curve with $\mathbf{A} = [a_i G]$
* Bob generates $r \leftarrow \texttt{rand()}$ and computes $\mathbf{A_r} = [r a_i G]$
* Alice gets $\mathbf{A_r}$, shuffles the elements into $\mathbf{A_{rs}}$, generates a $s \leftarrow \texttt{rand()}$ and elementwise multiplies $s \mathbf{A_{rs}}$
* Bob gets $s \mathbf{A_{rs}}$ and computes $r^{-1} \mod n$ where $n$ is the order of the curve
* Bob computes $(r^{-1} \mod n)s \mathbf{A_{rs}} = s \mathbf{A_s} = [s a_i G]$
* Alice computes $B = s b_i G$ and sends this to Bob
* Bob checks $A \cap B$, which will give him the total number of elements that intersect without knowing the items that intersect. Bob shares the intersection output but not $A_s$

In [50]:
#!pip install ecpy
from ecpy.curves import Curve
from Crypto.Util import number
import random
import numpy as np

In [10]:
def curvePick(x):
    """
    ['stark256',
 'frp256v1',
 'secp521r1',
 'secp384r1',
 'secp256k1',
 'secp256r1',
 'secp224k1',
 'secp224r1',
 'secp192k1',
 'secp192r1',
 'secp160k1',
 'secp160r1',
 'secp160r2',
 'Brainpool-p512t1',
 'Brainpool-p512r1',
 'Brainpool-p384t1',
 'Brainpool-p384r1',
 'Brainpool-p320t1',
 'Brainpool-p320r1',
 'Brainpool-p256r1',
 'Brainpool-p256t1',
 'Brainpool-p224r1',
 'Brainpool-p224t1',
 'Brainpool-p192r1',
 'Brainpool-p192t1',
 'Brainpool-p160r1',
 'Brainpool-p160t1',
 'NIST-P256',
 'NIST-P224',
 'NIST-P192',
 'Ed448',
 'Ed25519',
 'Curve448',
 'Curve25519']
    """
    curve = Curve.get_curve(Curve.get_curve_names()[x])
    return curve

In [94]:
curve = curvePick(4)
G = curve.generator
order = curve.order
a = [1, 44, 33, 10]
b = [60, 54, 19, 10]

assert all(a[i] > 0 for i in range(len(a))), "Must use positive numbers"
assert all(b[i] > 0 for i in range(len(b))), "Must use positive numbers"
 

s = number.getRandomNBitInteger(256)
r = number.getRandomNBitInteger(256)

raAi = [r * a[i] * G for i in range(len(a))]
sraAi = [s * raAi[i] for i in range(len(a))]

inv_r = pow(r, -1, order)

random.shuffle(sraAi)

sAi = [inv_r * sraAi[i] for i in range(len(a))]  
sBi = [s * b[i] * G for i in range(len(b))]
    
len(np.intersect1d([sBi[i].x for i in range(len(a))], [sAi[i].x for i in range(len(a))]))

1

# Oblivious transfer

[part 1](https://www.youtube.com/watch?v=HP0HnVmBs3g), and [part 2](https://www.youtube.com/watch?v=h6iiaAv_K0A) Even Goldreich Lempel 1985

* Alice has a public RSA key $(e,n)$ known to Bob and Alice private key is $(d,n)$
* Alice has two wire labels $(w_0,w_1)$ corresponding to the inputs to some gate
* Alice creates two random values $(r_0, r_1) \leftarrow \texttt{rand()}$
* Depending on what Bob wants, he has an input $b\leftarrow\{0,1\}$
* Based on $b$, Bob will either learn $r_0 \text{ or } r_1$
* Bob will pick a random value to blind the response wiht $r \leftarrow \texttt{rand()}$
* Bob computes $v = x_b + r^e \mod n$ and sends $v$ to Alice
* Alice computes
 1. $k_0 = (v-r_0)^d \mod n$
 2. $k_1 = (v-r_1)^d \mod n$
* Alice now sends over $(E(k_0,w_0), E(k_1,w_1))$

If we assume that $b=1$, we get that:
1. $k_0 = (r^e - r_0 + r_1)^d \mod n$
2. $k_1 = (r^e - r_1 + r_1)^d \mod n = r^{ed} \mod n = r$

Note that Alice cannot distinguish $(k_0, k_1)$ from random numbers. But $k_1$ in the example above is the $r$ Bob picked.


[Asecuritysite version](https://www.youtube.com/watch?v=hP22X0T_-s4a)

* Alice and Bob agree on generator $g$ and prime $p$
* Alice computes $A = g^a \mod n$
* Bob computes $B = g^b \mod n$
* Alice computes $s = B^a \mod n$
* Bob computes:
 1. If interested in record 0: $B = g^b \mod n$
 2. If interested in record 1: $B = A g^b \mod n$
 3. key $H(A^b)$
* Alice computes:
 1. $k_0 = H(B^a)$
 2. $k_1 = H((B/A)^a)$
* Alice sends over $(E(k_0,m_0), E(k_1, m_1))$

In [48]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib
import random

In [74]:
g = 9
n = 1001
c = 0
m0 = bytes('message zero', 'utf-8') #can also do m0.encode('utf8')
m1 = bytes('message one', 'utf-8')

a = random.randint(5,10)
b = random.randint(10,15)

Alice = pow(g,a,n)

Bob = (Alice * pow(g,b,n)) % n if c == 0 else pow(g,b,n)

# Alice computes
key0 = hashlib.sha256(bytes(str(pow(Bob,a,n)), 'utf-8')).digest()
key1 = hashlib.sha256(bytes(str(pow(Bob * pow(Alice, -1,n),a,n)), 'utf-8')).digest()

c0 = AES.new(key0, AES.MODE_ECB)
c1 = AES.new(key1, AES.MODE_ECB)

msg0 = c0.encrypt(pad(m0, 32))
msg1 = c1.encrypt(pad(m1, 32))

# Bob now
decipher0 = AES.new(key0, AES.MODE_ECB)
decipher1 = AES.new(key1, AES.MODE_ECB)

msg0_dec = decipher0.decrypt(msg0)
msg1_dec = decipher1.decrypt(msg1)

unpad(msg0_dec, 32) if c == 1 else unpad(msg1_dec, 32)

b'message one'

# COVID-19 Contact Tracker

saved as download

# GQ identification scheme

Saved as download

# RAPPOR

Bloom filter based privacy analytics

saved as download

# Infection Tracking The Epione Protocol

Relies on PSI

saved as download

# R1CS (Rank 1 Constraint System) in Go

saved as download

# zkSNARKS in Go

saved as download

# DLEQ 

code available at [git](https://github.com/dedis/kyber)

[paper is here](http://fc13.ifca.ai/proc/5-1.pdf)

We can use Non-interactive zero-knowledge (NIZK) proofs, and where Peggy (the 'Prover') just has to prove that she still knows her secret. For this Victor (the 'Verifier') can send Peggy a challenge, and where Peggy can prove that she can provide a solution for it. Peggy creates a secret value $(x)$ and then we creates two values $xG$ and $xH$, and can check that $\log_G(xG)==log_H(xH)$. Peggy first creates her secret $(x)$ from her password $(x=H(\texttt{Password}))$, and then calculates $xG$ and $xH$, and where $G$ and $H$ are two random points on an elliptic curve (in this case it is from an Edwards 25519 curve).

1. Peggy creates $x = H(\texttt{password})$ 
2. Peggy creates $(xG,xH)$ and checks that $\log_G(xG)==log_H(xH)$
3. Victor creates a random value $v \leftarrow \texttt(rand())$ and uses it to generate a challenge $c = H(xG,xH,vG,vH)$
4. Peggy responds with $r = v - cx$
5. The proof is (c,r,vG,vH)
6. Victor proves that Peggy knows $x$ using $vG == rG+c(xG)$ and $vH == rH + c(xH)$

The above works because $rG+c(xG)=(v-cx)G+c(xG)= vG-cxG+cxG=vG$

In [65]:
// Code from https://github.com/dedis/kyber
package main

import (
    "fmt"

    "go.dedis.ch/kyber"
    "go.dedis.ch/kyber/proof/dleq"
    "go.dedis.ch/kyber/suites"
    "go.dedis.ch/kyber/group/edwards25519"
    "go.dedis.ch/kyber/util/random"
    "crypto/sha256"
    "os"
    "errors"

)

var rng = random.New()

func NewDLEQProof(suite suites.Suite, G kyber.Point, H kyber.Point, x kyber.Scalar) (proof *dleq.Proof, xG kyber.Point, xH kyber.Point, v kyber.Scalar,err error) {
    // Encrypt base points with secret
    xG = suite.Point().Mul(x, G)
    xH = suite.Point().Mul(x, H)

    // Commitment
    v = suite.Scalar().Pick(suite.RandomStream())
    vG := suite.Point().Mul(v, G)
    vH := suite.Point().Mul(v, H)

    // Challenge
    h := suite.Hash()
    xG.MarshalTo(h)
    xH.MarshalTo(h)
    vG.MarshalTo(h)
    vH.MarshalTo(h)
    cb := h.Sum(nil)
    c := suite.Scalar().Pick(suite.XOF(cb))

    // Response
    r := suite.Scalar()
    r.Mul(x, c).Sub(v, r)

    return &dleq.Proof{c, r, vG, vH}, xG, xH, v,nil
}

func Verify(p *dleq.Proof, suite suites.Suite, G kyber.Point, H kyber.Point, xG kyber.Point, xH kyber.Point) error {
    rG := suite.Point().Mul(p.R, G)
    rH := suite.Point().Mul(p.R, H)
    cxG := suite.Point().Mul(p.C, xG)
    cxH := suite.Point().Mul(p.C, xH)
    a := suite.Point().Add(rG, cxG)
    b := suite.Point().Add(rH, cxH)
    if !(p.VG.Equal(a) && p.VH.Equal(b)) {
        return  errors.New("invalid proof")
    }
    return nil
}


func main() {



    suite := edwards25519.NewBlakeSHA256Ed25519()

    m:="Hello"

    argCount := len(os.Args[1:])

        if (argCount>0) {m= string(os.Args[1])}

    message := []byte(m)
    scal := sha256.Sum256(message[:])

    x := suite.Scalar().SetBytes(scal[:32])

    g := suite.Point().Pick(rng)
    h := suite.Point().Pick(rng)

    proof, xG, xH, v, err := NewDLEQProof(suite, g, h, x)

    if (err==nil) {
        fmt.Printf("Secret: %s\n",m) 
        fmt.Printf("x: %x\n\n",scal) 
        fmt.Printf("G: %s\nH: %s\n",g,h) 
        fmt.Printf("xG %s\nxH %s\n",xG,xH) 
        
 
        fmt.Printf("\nProof created\nChallenge (c): %s\nResponse (r): %s\nvG: %s\nvH: %s\nv: %s",proof.C,proof.R,proof.VG,proof.VH,v) 
    }

    rtn:=Verify(proof,suite, g, h, xG, xH)


    if (rtn==nil) { fmt.Printf("\n\nPeggy has proven she still knows her secret") }

}

SyntaxError: invalid syntax (<ipython-input-65-b4d24f6f9bf7>, line 1)

In [66]:
Secret: Testing 123
x: 9734aef6d3788ba985e78f7b3785dc4817e770be92a4e5e57e64a92cc9c2fc25

G: 5c72b580b791ee5a699c71dcb40d417850eef748b2812dad0413a2d7b962e15b
H: 9c26104e3904e7a60a57492080c00218c4fa71de4ca291a59585a681e378e463
xG a36bbfe5e20f41da437c03fa7f7864bf02d7dee029732f826dc172d36aa6706a
xH ce37afe095f099d0f1cbcf41e11ba6abad716426ac49c044bdbce343f0c000bb

Proof created
Challenge (c): 4e65bfc0346a049d8ebec91e47693d51565f00719e842412a832ef582b92500c
Response (r): 307ede7a7d0a3507512f3963a7d2e91436c077085455fe9b016731214832320e
vG: 10fa72e0636b63593ffb8d9ada2d2960adc7b297f8b45730d5334e8339576b15
vH: 320041b2907ab8b6f84dc7566ee58a449d3bd19d02a750129c9eb785daff6f8e
v: 9cc4bb3a257874799514d5380227919c7b52a13c11777f2de8fac46fb796e00a

Peggy has proven she still knows her secret

SyntaxError: invalid syntax (<ipython-input-66-ea594a34ea00>, line 1)

# Dragonfly

Dragonfly is in principle an authenticated DH key exchange. The Dragonfly zero-knowledge proof method is used in WPA-3 for Simultaneous Authentication of Equals (SAE)

* Alice generates $(a,A) \leftarrow \texttt{rand()}$
* Alice computes $sA = (a+A) \mod q$ where $q \in \mathbb{P}$
* Alice computes $PE^{-A}$ where $PE$ is the Password Equivalent (e.g., a hash valued of a password that Bob and Alice know)
* Alice sends to Bob $(sA,PE^{-A})$

* Bob generates $(b,B) \leftarrow \texttt{rand()}$
* Bob computes $sB = (b+B) \mod q$ where $q \in \mathbb{P}$
* Bob computes $PE^{-B}$
* Bob sends to Alice $(sB,PE^{-B})$

Either one can now derive a shared secret by:

* Alice computes $s = (PE^{sB} \times PE^{-B})^a$
* Bob computes $s = (PE^{sA} \times PE^{-A})^b$
* $s$ acts as a long term key that can be used for the $\verb@confirm@$ phase, as follows:
 1. KCK stands for key expansion of long term key.
 2. $\texttt{Confirm-A} = H(KCK | \texttt{scalar}A | a | \texttt{element}A | \texttt{element}B)$ Alice sends to Bob
 3. $\texttt{Confirm-B} = H(KCK | \texttt{scalar}B | b | \texttt{element}B | \texttt{element}A)$ Bob sends to Alice
 4. Exchange new master key to create session key

In [64]:
# Dragonfly
import hashlib
import random
import sys
import libnum




q=131
text="hello"

a=random.randint(1,1000)
b=random.randint(1,1000)
A=random.randint(1,1000)
B=random.randint(1,1000)


sA= a + A
sB = b + B



PE = int(hashlib.md5(text.encode()).hexdigest()[:8], 16)

elementA = libnum.invmod(pow(PE,A),q)
elementB = libnum.invmod(pow(PE,B),q)

PEsA = pow(PE,sA,q)
PEsB = pow(PE,sB,q)

print("Password:",text)
print("Alice generates two random values")
print("a:\t",a)
print("A:\t",A)
print("\nBob generates two random values")
print("b:\t",b)
print("B:\t",B)

print("\nAlice calculates elementA")
print("Element A:",elementA)
print("\nBob calculates elementB")
print("Element B:",elementB)


ss1 = pow(PEsA * elementA,b,q)
ss2 = pow(PEsB * elementB,a,q)

print("\nThey exchange values and calculate a secret share")
print("\nAlice share:\t",ss1)
print("Bob share:\t",ss2)

Password: hello
Alice generates two random values
a:	 470
A:	 997

Bob generates two random values
b:	 242
B:	 453

Alice calculates elementA
Element A: 82

Bob calculates elementB
Element B: 118

They exchange values and calculate a secret share

Alice share:	 84
Bob share:	 84


# Chaim-Pedersen method of ZKP

1. Peggy and Victor agree on $(g, g^a, g^b,g^{ab})$ and modulus $q$
2. Peggy generates $r \leftarrow \texttt{rand()}$
3. Peggy sends $y_1 = g^r, y_2 = g^{br}$
4. Victor generates random challenge $s\leftarrow \texttt{rand()}$
5. Peggy computes $z=r+as \mod q$
6. Victor checks in mod q that:
 * $g^z = g^{as} y_1$
 * $g^{bz} = g^{abs}y_2$

In [62]:
import random

q=10009

s=random.randint(1,1000)
r=random.randint(1,1000)

g=3
a=10
b=13
A=pow(g,a,q)
B=pow(g,b, q)
C=pow(g,(a*b),q)

y1=pow(g,r,q)
y2=pow(B,r,q)

z=(r+a*s) % q


print("Victor and Peggy agree of (g,g^a, g^b and g^ab) =(",g,A,B,C,")")
print("\nPeggy generates random number (r)",r)
print("Peggy sends y1 (g^r, B^r)=(",y1,y2,")")

print()


print("Victor sends a challenge (s)=",s)

print("Peggy computes z=r+as (mod q)=",z)

print("\nVictor now checks these are the same")
print("Victor checks g^z=",pow(g,z,q))

print("Victor checks A^s y1=",(A**s * y1) % q)

print("\nVictor now checks these are the same")
print("Victor checks B^z=", pow(B,z,q))
print("Victor checks C^s y2=",(C**s * y2) % q)

Victor and Peggy agree of (g,g^a, g^b and g^ab) =( 3 9004 2892 5980 )

Peggy generates random number (r) 277
Peggy sends y1 (g^r, B^r)=( 9334 2611 )

Victor sends a challenge (s)= 469
Peggy computes z=r+as (mod q)= 4967

Victor now checks these are the same
Victor checks g^z= 1739
Victor checks A^s y1= 1739

Victor now checks these are the same
Victor checks B^z= 1724
Victor checks C^s y2= 1724


# Diffie-Hellman with ZKP

#!pip install pyDH

See [git](https://github.com/tinydeltas/zkp-dh/blob/master/zkp-dh.py)
See also [paper](ftp://ftp.inf.ethz.ch/pub/crypto/publications/CamSta97b.pdf)

* Bob generates $x$
* Alice generates $y$
* $B = G ^ x \mod p$
* $A = G ^ y \mod p$
* $A^x \mod p == B^y \mod p$
* Alice challenges Bob to prove knowledge of $x$

In [60]:
secret = self.get_shared_secret(remote_pub)

        # Random key in the group Z_q
        randKey = DiffieHellman() # random secret
        commit1 = randKey.public
        commit2 = randKey.get_shared_secret(remote_pub)

        # shift and hash
        concat = str(G) + str(prover_pub) + str(remote_pub) + str(secret) + str(commit1) + str(commit2)
        h = hashlib.md5()
        h.update(concat.encode("utf-8"))
        challenge = int(h.hexdigest(), 16)
        product = (self.secret * challenge) % phi
        response = (randKey.secret - product) % phi

IndentationError: unexpected indent (<ipython-input-60-30a4f9350f9e>, line 4)

and where the challenge is a hash of G, the provers public key, the remote public key, the secret key, a random value and a shared secret random value. PHI is p-1.

In this case secret will be the shared secret that has been negotiated by Bob and Alice.

# Pedersen Commitment

1. Take two large primes $(p,q)$
2. Create a generator value $g$ with order $q$ and a subgroup of $Z^*_p$
3. Compute $h = g^s \mod p$ where $s \in Z_q$
4. Sender creates commitment for msg $m$ and a random number $r$ by $c = g^m h^r \mod p$
5. Sender reveals $(m,r)$ to prove commitment

In [1]:
from Crypto import Random
from Crypto.Util import number

In [37]:
g = 3
q = 0
while number.isPrime(q) == 0:
    p = number.getPrime(16)
    q = p * 2  + 1

s = number.getRandomRange(1,q-1)
h = pow(g,s,q)

# Sender now creates commitment for msg
m0 = number.getRandomNBitInteger(32)
r0 = number.getRandomNBitInteger(32)
c0 = (pow(g,m0,q) * pow(h,r0,q)) % q
m1 = number.getRandomNBitInteger(32)
r1 = number.getRandomNBitInteger(32)
c1 = (pow(g,m1,q) * pow(h,r1,q)) % q

print('Commitment is c:', c0)
print('\nPeggy can reveal r and m at a later time')
m0+m1 %q, c0*c1 %q

Commitment is c: 67683

Peggy can reveal r and m at a later time


(3712291362, 3722)

In [38]:
from Crypto import Random
from Crypto.Util import number
import sys

def generate(param):
        q = param[1]
        g = param[2]
        h = param[3]
        return q,g,h

class verifier:
    def setup(self, security):
        p = number.getPrime(2 * security, Random.new().read)
        q = 2*p + 1


        g = number.getRandomRange(1, q-1)
        s = number.getRandomRange(1, q-1)
        print("Secret value:\t",s)
        h = pow(g,s,q)
        
        param = (p,q,g,h)
        print("p=",p)
        print("q=",q)
        print("g=",g)
        print("h=",h)

        return param

    def open(self, param, c, x, *r):
        result = "False"
        q,g,h = generate(param)

        sum = 0
        for i in r:
            sum += i

        res = (pow(g,x,q) * pow(h,sum,q)) % q

        if(c == res):
            result = "True"
        return result  

    def add(self, param, *cm):
        addCM = 1
        for x in cm:
            addCM *= x
        addCM = addCM % param[1]
        return addCM
        
class prover: 
    def commit(self, param, x):
        q,g,h = generate(param)
        
        r = number.getRandomRange(1, q-1)
        c = (pow(g,x,q) * pow(h,r,q)) % q
        return c, r
    

security = 80
msg1 = 1
msg2 = 2

if (len(sys.argv)>1):
	msg1=int(sys.argv[1])

if (len(sys.argv)>2):
	msg2=int(sys.argv[2])

v = verifier()
p = prover()

param = v.setup(security)

c1, r1 = p.commit(param, msg1)
c2, r2 = p.commit(param, msg2)

addCM = v.add(param, c1, c2)

print("\nMsg1:",msg1)
print("Msg2:",msg2)

print("\nc1,r1:",c1,",",r1)
print("c2,r2:",c2,",",r2)
print("\nWe can now multiply c1 and c2, which is same as adding Msg1 and Msg2")
print("\nCommitment of adding (Msg1+Msg2):\t",addCM)

result1 = v.open(param, c1, msg1, r1)
result2 = v.open(param, c2, msg2 , r2)

print("\nResult of verifying c1:\t\t",result1)
print("Result of verifying c2:\t\t",result2)

result = v.open(param, addCM, msg1 + msg2 , r1, r2)

print("Result of verify Msg+Msg2:\t",result)

ValueError: invalid literal for int() with base 10: '-f'

# Proving password in ZK

* We want Peggy to generate a ZK proof of knowledge of password and publish this proof. 
* Victor issues a challenge tied to the ZKP of password to Peggy.
* Peggy now proves that she still knows the committed password.

Many ways exist to solve the above, but one commonly used method is the "non-interactive random oracle access" for ZKP defined as the Fiat-Shamir heuristic. The process relies on the discrete log problem that is easy for Peggy to prove, but computationally non efficient for Eve to prove, and efficient for Victor to verify. The method is as follows (all operations are done in mod p):

1. Victor and Peggy agree on a generator $g$ and a prime number $p$.
2. Peggy picks a password and uses a cryptographic hashing algorithm $H()$ to generate a digest of her password.
3. Peggy does: $x=\texttt{int(H(password))}$
4. Peggy sends: $y = g^x \mod p$
5. When Peggy wants to log on, she generates a random value $v \leftarrow \texttt{rand()}$ and computes $t = g^v \mod p$
6. Victor sends Peggy a challenge $c \leftarrow \texttt{rand()}$ 
7. Peggy computes $r = v - c \times t \mod p$ and sends $r$ to Victor
8. Victor computes $g^ry^c \mod p$ and checks if it equals $t$.

This works because:

* Peggy cannot generate $(y,t,r)$ without knowing $(x,v,c)$
* Victor sends a public challenge $c$
* Eve cannot generate $(y,r)$ as she does not know $x$ and cannot solve discrete log efficiently
* Since $g^r y^c = g^{v-cx}(g^{x})^c = g^v = t$, assuming $\mod p$, Victor can verify that Peggy indeed knows $x$

In [31]:
from Crypto.Util import number
from Crypto.Hash import SHA256

In [47]:
g, p = 3, pow(2,19)-1
password = "Thisisabadpassword"
x = SHA256.new(bytes(password, 'utf-8'))
x = int(x.hexdigest(), 16)
y = pow(g, x, p) # Peggy sends this to Victor

# To authenticate, Peggy generates t = g ** v and sends this as auth request to Victor
v = number.getRandomNBitInteger(64)
t = pow(g, v, p)

# Victor generates c and sends c to Peggy, who returns r = v - c*x. 
c = number.getRandomNBitInteger(64) # Public
r = v - c * x # Peggy computes and makes public

# Victor computes
verify = (pow(g, r, p) * pow(y, c , p)) % p
verify == t

True

# Proof of knowledge in ZK with Go

[logical one](https://asecuritysite.com/encryption/go_proof2)

Within proof of knowledge, Peggy (the prover) must verify to Victor (the verifier) that she can solve a difficult problem. We must then have: An honest prover who can verify themselves to the verifier (completeness); A high probability that prover cannot cheat, and thus guess the solution (soundness); The prover cannot guess the secret from the proof (zero knowledge, witness hiding and minimum-disclosure). A standard proof in discrete logs, is to show that Peggy still knows the value of x given that the prover knows (X) and where q is a prime number: $X=g^x \mod q$. This page does two valid proofs and one invalid one.

Same base point and values as for the logical proof below

In [44]:
package main

import (
    "fmt"
    "go.dedis.ch/kyber/v3"
    "go.dedis.ch/kyber/v3/group/edwards25519"
    "go.dedis.ch/kyber/v3/proof"
    "encoding/hex"
)

func main() {
    
    
    suite := edwards25519.NewBlakeSHA256Ed25519()
    rand := suite.RandomStream()

    x := suite.Scalar().Pick(rand)
    y := suite.Scalar().Pick(rand)
    B := suite.Point().Base()
    X := suite.Point().Mul(x, nil)
    Y := suite.Point().Mul(y, nil)
    R := suite.Point().Add(X, Y)
    
    
    fmt.Printf("X=xB and Y=yB and R=xB+yB\n")
    fmt.Printf("x=%s, B=%s, X=%s, y=%s, Y=%s, R=%s\n\n",x,B,X,y,Y,R)

    // X = xB
    pred := proof.Rep("X", "x", "B")
    fmt.Println(pred.String())


    sval := map[string]kyber.Scalar{"x": x}
    pval := map[string]kyber.Point{"B": B, "X": X}

    prover := pred.Prover(suite, sval, pval, nil)
    proof_, _ := proof.HashProve(suite, "TEST", prover)

    fmt.Printf("We need to prove that we known the discrete log of X\n")
    fmt.Print("Proof:\n" + hex.Dump(proof_))

    
    // Verify this knowledge proof.
    verifier := pred.Verifier(suite, pval)
    err := proof.HashVerify(suite, "TEST", verifier, proof_)
    if err != nil {
        fmt.Println("Proof failed to verify: ", err)
        return
    }
    fmt.Println("-- Proof verified.\n")


    pred = proof.Rep("R", "x", "B","y", "B")
    fmt.Println(pred.String())

    sval = map[string]kyber.Scalar{"x": x,"y": y}
    pval = map[string]kyber.Point{"B": B, "R": R}
    prover = pred.Prover(suite, sval, pval, nil)
    proof_, _ = proof.HashProve(suite, "TEST", prover)
    fmt.Print("Proof:\n" + hex.Dump(proof_))
    verifier = pred.Verifier(suite, pval)
    err = proof.HashVerify(suite, "TEST", verifier, proof_)
    if err != nil {
        fmt.Println("Proof failed to verify: ", err)
        return
    }

    fmt.Println("-- Proof verified.\n")


 //   pred = proof.Rep("Y", "y", "B") - Correct
// This test will fail as we try to prove X=yB
     pred = proof.Rep("X", "y", "B") - Incorrect
    fmt.Println(pred.String())

    sval = map[string]kyber.Scalar{"y": y}
    pval = map[string]kyber.Point{"B": B, "Y": Y}
    prover = pred.Prover(suite, sval, pval, nil)
    proof_, _ = proof.HashProve(suite, "TEST", prover)
    fmt.Print("Proof:\n" + hex.Dump(proof_))
    verifier = pred.Verifier(suite, pval)
    err = proof.HashVerify(suite, "TEST", verifier, proof_)
    if err != nil {
        fmt.Println("Proof failed to verify: ", err)
        return
    }

    fmt.Println("-- Proof verified.")
}

SyntaxError: invalid syntax (<ipython-input-44-cf5ecce8ad4d>, line 1)

In [45]:
X=xB and Y=yB and R=xB+yB
x=6f3632050865e77afdb974d58b357bce3feb94a0056380687c3c32b070c03f05, B=5866666666666666666666666666666666666666666666666666666666666666, X=f51db694db6a19ffe9970251f8f909fe571b4be79cf84421ee388812922f0540, y=b5b295dcc4c49ee7798ab2513ae3b01711a3d875cccd694f3ca593db0731d606, Y=a97c6ec8de585b34ce837f009ce5e999225accd50b8a95cb9d01c8a287a3de8b, R=4acfc0b4a6afc7698b3ccd74a358cc25d051e903fbe60e64ab8fa5ec97d99383

X=x*B
We need to prove that we known the discrete log of X
Proof:
00000000  10 50 30 e2 ff d1 eb e9  c3 a8 55 1d 97 c0 1f 65  |.P0.......U....e|
00000010  a0 07 bc 13 84 55 b6 2a  0e 56 bc f5 04 be 10 3d  |.....U.*.V.....=|
00000020  68 bc 1a 15 b3 0b 94 e0  2e eb 5b 09 b4 05 86 c4  |h.........[.....|
00000030  aa 64 3c f4 b6 5a 09 e3  72 fd 41 83 34 5d db 03  |.d<..Z..r.A.4]..|
-- Proof verified.

R=x*B+y*B
Proof:
00000000  33 bb b9 a4 96 ea 46 15  b6 8c c5 7d 4a 76 fd 9c  |3.....F....}Jv..|
00000010  1b fd 13 2d a5 0e b8 05  f5 ad c1 be c0 5a ce 64  |...-.........Z.d|
00000020  7d bc de 3a eb d7 47 7b  2e e2 5c c9 cb a2 0a fa  |}..:..G{..\.....|
00000030  40 8b 8f e2 82 ea 7a d6  81 bb 87 5c b4 bf 68 00  |@.....z....\..h.|
00000040  a0 f5 e1 0e 63 ae bf ec  ce b0 87 26 9a c6 59 29  |....c......&..Y)|
00000050  10 f6 6f 46 7b 2e 6a da  3f 9e be c3 70 ec 01 00  |..oF{.j.?...p...|
-- Proof verified.

Y=y*B
Proof:
00000000  27 b9 0c d3 59 95 19 00  62 cc e4 0e 34 c0 49 f6  |'...Y...b...4.I.|
00000010  14 5e bb 57 09 58 d7 bf  f8 0a e7 7d 40 d9 1c ac  |.^.W.X.....}@...|
00000020  ba 83 2e c6 d8 e2 8a cf  06 0a de 91 52 10 1a df  |............R...|
00000030  14 58 cb 0f e6 ca 9f bd  b7 26 60 c3 ae 50 e7 00  |.X.......&`..P..|
-- Proof verified.

SyntaxError: cannot assign to operator (<ipython-input-45-d42ddfd2cf7a>, line 1)

# Logical proofs in ZK with Go

A standard proof in discrete logs, is to show that Peggy still knows the value of $x$ given that the prover knows $(X)$ and where $q$ is a prime number: $X=gx \mod q$. This page uses predicates of $\texttt{X=x*B && Y=y*B}$ (which is true) and $\verb@X=x*B && Y=x*B@$ (which is false).

We need to create:

1. $X = xB$
2. $Y=yB$
3. $R=(x+y)B$

where $B$ is a base point on the elliptic curve and where $(x,y)$ are secrets

We see the value of "5866666666666666666666666666666666666666666666666666666666666666" for the base point. This is the base point for the Edwards Curve 25519 (−x^2+y^2=1−(121665/121666)×x^2×y^2). The prime number used is q=2^255−19 and which is "57896044618658097711785492504343953926634992332820282019728792003956564819949".

In [18]:
package main

import (
    "fmt"
    "go.dedis.ch/kyber"
    "go.dedis.ch/kyber/group/edwards25519"
    "go.dedis.ch/kyber/proof"
    "encoding/hex"
)

func main() {
    
    
    suite := edwards25519.NewBlakeSHA256Ed25519()
    rand := suite.RandomStream()

    x := suite.Scalar().Pick(rand)
    y := suite.Scalar().Pick(rand)
    B := suite.Point().Base()
    X := suite.Point().Mul(x, nil)
    Y := suite.Point().Mul(y, nil)
    R := suite.Point().Add(X, Y)
    
    
    fmt.Printf("X=xB, Y=yB and R=xB+yB\n")
    fmt.Printf("B=%s, x=%s, X=%s, y=%s, Y=%s, R=%s\n\n",B,x,X,y,Y,R)


    sval := map[string]kyber.Scalar{"x": x,"y":y}
    pval := map[string]kyber.Point{"B": B, "X": X, "Y": Y}



    // X = xB
    pred1 := proof.Rep("X","x","B")
    pred2 := proof.Rep("Y", "y", "B")

    pred := proof.And(pred1, pred2)

    fmt.Println(pred.String())

    prover := pred.Prover(suite, sval, pval, nil)
    proof_, _ := proof.HashProve(suite, "TEST", prover)

    fmt.Printf("We need to prove that we known the discrete log of X\n")
    fmt.Print("Proof:\n" + hex.Dump(proof_))

    
    // Verify this knowledge proof.
    verifier := pred.Verifier(suite, pval)
    err := proof.HashVerify(suite, "TEST", verifier, proof_)
    if err != nil {
        fmt.Println("Proof failed to verify: ", err)
        return
    }
    fmt.Println("-- Proof verified.\n")




    // X = xB

    pred1 = proof.Rep("X","x","B")
    pred2 = proof.Rep("Y", "x", "B")


    pred = proof.And(pred1, pred2)

    fmt.Println(pred.String())

    prover = pred.Prover(suite, sval, pval, nil)
    proof_, _= proof.HashProve(suite, "TEST", prover)

    fmt.Printf("We need to prove that we known the discrete log of X\n")
    fmt.Print("Proof:\n" + hex.Dump(proof_))

    
    // Verify this knowledge proof.
    verifier = pred.Verifier(suite, pval)
    err = proof.HashVerify(suite, "TEST", verifier, proof_)
    if err != nil {
        fmt.Println("Proof failed to verify: ", err)
        return
    }
    fmt.Println("-- Proof verified.\n")
    
    

}

SyntaxError: invalid syntax (<ipython-input-18-1c4b7f821954>, line 1)

In [19]:
X=xB, Y=yB and R=xB+yB
B=5866666666666666666666666666666666666666666666666666666666666666, x=2b2b55e63072a70de05e93a4bb7aa4157566e78fce642f89a5919a9f147e440e, X=681fd35655e161a0d2e8fe97d2dd14763d4f5a70ff32dfd37adfe38a010f9eae, y=f8424eaeffa9d7b6e009e01c7c0730ec0702cb25576b2f7e254f10d060175108, Y=8f92691995c27fc50f89bc7df5ea3c5807c871b502bafcddb71ba2761a7e7eba, R=e3298170d7178d953db51012200e77e4c3b572d9389f91c7aeb31e297dfc653e

X=x*B && Y=y*B
We need to prove that we known the discrete log of X
Proof:
00000000  01 e7 d4 c9 1e 9b 53 22  e8 fb f0 75 bb f7 1c 07  |......S"...u....|
00000010  83 23 33 6e f2 5e 62 56  21 39 65 09 1a 2f 42 28  |.#3n.^bV!9e../B(|
00000020  01 9b 45 7f 52 9f 69 0c  b3 b3 76 fb 6b 00 a3 15  |..E.R.i...v.k...|
00000030  20 5b 8a 88 da e2 a2 dd  3c f7 2f c8 7c ec 87 23  | [......../.|..#|
00000040  25 90 14 c9 59 1f 57 2b  7e d4 9c a4 cc ec 6d 2a  |%...Y.W+~.....m*|
00000050  bd 29 b6 86 2a 5c b0 a6  da e0 50 dc b8 88 78 0e  |.)..*\....P...x.|
00000060  f7 54 c0 37 0e fb 2f 67  5e ad ef b3 ea c2 56 f5  |.T.7../g^.....V.|
00000070  57 55 ca 49 38 f8 23 08  fc 28 2e b3 2c 30 a7 0a  |WU.I8.#..(..,0..|
-- Proof verified.

X=x*B && Y=x*B
We need to prove that we known the discrete log of X
Proof:
00000000  c2 eb 00 82 0d 20 57 b7  a5 18 09 43 c8 36 98 05  |..... W....C.6..|
00000010  97 06 e5 81 50 42 cd c2  b7 d3 dc ba dd 9a 96 87  |....PB..........|
00000020  c2 eb 00 82 0d 20 57 b7  a5 18 09 43 c8 36 98 05  |..... W....C.6..|
00000030  97 06 e5 81 50 42 cd c2  b7 d3 dc ba dd 9a 96 87  |....PB..........|
00000040  04 9b d6 97 2e 81 64 b6  1f 3c e5 be 79 f1 a9 ea  |......d.....y...|
00000050  77 64 10 96 f3 72 b5 22  2d 12 f5 92 61 96 27 0f  |wd...r."-...a.'.|
Proof failed to verify:  invalid proof: commit mismatch

SyntaxError: cannot assign to operator (<ipython-input-19-451dfbffc9e0>, line 1)

# Wesolowski Proof of Exponent

[source](https://eprint.iacr.org/2018/623.pdf)

Alice generates a proof that she knows the required exponentiation.

1. Shared values $(u,x,w)$
2. Alice must prove she knows $w=u^x$
3. Bob generates an $n$-bit prime $p$ and sends it to Alice
4. Bob computes a quotient $q = x / p$ and a residue $x = qp + r$
5. Bob sends $Q=u^q$
6. Bob computes $r=x \mod p$
7. Bob accepts the proof if $Q^p u ^r = w$

In [1]:
from Crypto.Util import number

In [17]:
bits = 32
p = number.getPrime(bits) # Generated by verifier
u = number.getRandomNBitInteger(256)
x = number.getRandomNBitInteger(256)
w = pow(u,x,p)

print('Shared values', (u,x,w))

q = x // p
r = (x - q * p) % p

print('Verifier sends p=%d and prover uses x to compute q = %d and r = %d' % (p,q,r))

Q = pow(u,q,p)

print('Prover sends', Q)

print('Verifier computes r = x mod p:', (x % p))

print('Verifier checks that Q^p mod p == w:', w == (pow(Q,p,p) * pow(u,r,p)) % p)

Shared values (93467751359113378278996261055180912782652723097655468569718782208186866539431, 76506396169990552718967481412307134726825780531502619318524178564417756234322, 1291136727)
Verifier sends p=2235264923 and prover uses x to compute q = 34226992685640847735419629010259887985020637543240604647793767828216 and r = 641766954
Prover sends 1695728632
Verifier computes r = x mod p: 641766954
Verifier checks that Q^p mod p == w: True


# ZK Bullet proofs

* [soure](https://web.stanford.edu/~buenz/pubs/bulletproofs.pdf) 
* [iacr](https://eprint.iacr.org/2017/1066.pdf)
* Simple [git](https://github.com/omershlo/simple-bulletproof-js)
* [Monero implementation](https://ostif.org/wp-content/uploads/2018/10/OSTIF-QuarksLab-Monero-Bulletproofs-Final2.pdf)

* Provides proof that we know $x$ where $x \in [0, 2^n - 1]$, where $n$ is bit length.
* If $n = 8$ we can generate proof that $x \in [0,255]$

Method

1. Secret value $x$, two elliptic curve points $(H,G)$, a random value $r$
2. Generate the public key point $T_x = xH$. Note that small $x$ can be easily brute forced.
3. The Pedersen Commit becomes $C(x,r) = xH + rG$
4. $x$ cannot be determined without knowing $r$.

Transaction

* Transaction value $v$ used to define point $v H$
* Multiple transactions values $(v_0, v_1, v_2)$ can be summed to $(v_0 + v_1+ v_2)H$
* $(r_0 G + v_0 H) + (r_1 G + v_1 H) = ((r_0 + r_1)G + (v_0 + v_1)H)$
* Used to prove that inputs equal outputs
* Within Bulletproofs we can implement range proofs $x\in[0.2^n-1]$

The public parameters are [1]:

l: cardinality of the subgroup of the elliptic curve used (Ed25519)

N: bitsize of the elements whose range one wants to prove (N = 64)

M: number of proofs to aggregate

G: the base point of the subgroup of the elliptic curve used

H: another generator of the subgroup of the elliptic curve used whose discrete log wrt G is not known and hard to find

Gi: a list of M*N generators of the subgroup of the elliptic curve used whose discrete log wrt any other generator is not known and hard to find

Hi: a list of M*N generators of the subgroup of the elliptic

The committed to values are [1]:

* v: the values to hide (and are in the range 0≤vj≤2N)
* gamma: hiding values

The bulletproof is [1]:

V: a vector of curve points, Pedersen commitments to v[i] with hiding values gamma[i]

A: a curve point, vector commitment to aL and aR with hiding value alpha

S: a curve point, vector commitment to sL and sR with hiding value rho

T1: a curve point, Pedersen commitment to t1 with hiding value tau1

T2: a curve point, Pedersen commitment to t2 with hiding value tau2

taux: a scalar, hiding value related to T1, T2, V and t

mu: a scalar, hiding value related to A and S

L: a vector of curve points of size log2(M*N) computed in the inner product protocol

R: a vector of curve points of size log2(M*N) computed in the inner product protocol

a: a scalar computed in the inner product protocol

b: a scalar computed in the inner product protocol

t: a scalar, inner product value to be verified

In [ ]:
package main

import (
    "crypto/elliptic"
    "crypto/rand"
    "crypto/sha256"
    "encoding/binary"
    "math/big"
    "os"
    "fmt"
    "github.com/btcsuite/btcd/btcec"
    "math"
    "strconv"
)

var EC CryptoParams
var VecLength = 8

type ECPoint struct {
    X, Y *big.Int
}

// Equal returns true if points p (self) and p2 (arg) are the  same.
func (p ECPoint) Equal(p2 ECPoint) bool {
    if p.X.Cmp(p2.X) == 0 && p2.Y.Cmp(p2.Y) == 0 {
        return true
    }
    return false
}

// Mult multiplies point p by scalar s and returns the resulting point
func (p ECPoint) Mult(s *big.Int) ECPoint {
    modS := new(big.Int).Mod(s, EC.N)
    X, Y := EC.C.ScalarMult(p.X, p.Y, modS.Bytes())
    return ECPoint{X, Y}
}

// Add adds points p and p2 and returns the resulting point
func (p ECPoint) Add(p2 ECPoint) ECPoint {
    X, Y := EC.C.Add(p.X, p.Y, p2.X, p2.Y)
    return ECPoint{X, Y}
}

// Neg returns the additive inverse of point p
func (p ECPoint) Neg() ECPoint {
    negY := new(big.Int).Neg(p.Y)
    modValue := negY.Mod(negY, EC.C.Params().P) // mod P is fine here because we're describing a curve point
    return ECPoint{p.X, modValue}
}

type CryptoParams struct {
    C   elliptic.Curve      // curve
    KC  *btcec.KoblitzCurve // curve
    BPG []ECPoint           // slice of gen 1 for BP
    BPH []ECPoint           // slice of gen 2 for BP
    N   *big.Int            // scalar prime
    U   ECPoint             // a point that is a fixed group element with an unknown discrete-log relative to g,h
    V   int                 // Vector length
    G   ECPoint             // G value for commitments of a single value
    H   ECPoint             // H value for commitments of a single value
}

func (c CryptoParams) Zero() ECPoint {
    return ECPoint{big.NewInt(0), big.NewInt(0)}
}

func check(e error) {
    if e != nil {
        panic(e)
    }
}

/*
Vector Pedersen Commitment
Given an array of values, we commit the array with different generators
for each element and for each randomness.
*/

/*
Two Vector P Commit
Given an array of values, we commit the array with different generators
for each element and for each randomness.
*/
func TwoVectorPCommit(a []*big.Int, b []*big.Int) ECPoint {
    if len(a) != len(b) {
        fmt.Println("TwoVectorPCommit: Uh oh! Arrays not of the same length")
        fmt.Printf("len(a): %d\n", len(a))
        fmt.Printf("len(b): %d\n", len(b))
    }

    commitment := EC.Zero()

    for i := 0; i < EC.V; i++ {
        commitment = commitment.Add(EC.BPG[i].Mult(a[i])).Add(EC.BPH[i].Mult(b[i]))
    }

    return commitment
}

/*
Vector Pedersen Commitment with Gens
Given an array of values, we commit the array with different generators
for each element and for each randomness.
We also pass in the Generators we want to use
*/
func TwoVectorPCommitWithGens(G, H []ECPoint, a, b []*big.Int) ECPoint {
    if len(G) != len(H) || len(G) != len(a) || len(a) != len(b) {
        fmt.Println("TwoVectorPCommitWithGens: Uh oh! Arrays not of the same length")
        fmt.Printf("len(G): %d\n", len(G))
        fmt.Printf("len(H): %d\n", len(H))
        fmt.Printf("len(a): %d\n", len(a))
        fmt.Printf("len(b): %d\n", len(b))
    }

    commitment := EC.Zero()

    for i := 0; i < len(G); i++ {
        modA := new(big.Int).Mod(a[i], EC.N)
        modB := new(big.Int).Mod(b[i], EC.N)

        commitment = commitment.Add(G[i].Mult(modA)).Add(H[i].Mult(modB))
    }

    return commitment
}

// The length here always has to be a power of two
func InnerProduct(a []*big.Int, b []*big.Int) *big.Int {
    if len(a) != len(b) {
        fmt.Println("InnerProduct: Uh oh! Arrays not of the same length")
        fmt.Printf("len(a): %d\n", len(a))
        fmt.Printf("len(b): %d\n", len(b))
    }

    c := big.NewInt(0)

    for i := range a {
        tmp1 := new(big.Int).Mul(a[i], b[i])
        c = new(big.Int).Add(c, new(big.Int).Mod(tmp1, EC.N))
    }

    return new(big.Int).Mod(c, EC.N)
}

func VectorAdd(v []*big.Int, w []*big.Int) []*big.Int {
    if len(v) != len(w) {
        fmt.Println("VectorAdd: Uh oh! Arrays not of the same length")
        fmt.Printf("len(v): %d\n", len(v))
        fmt.Printf("len(w): %d\n", len(w))
    }
    result := make([]*big.Int, len(v))

    for i := range v {
        result[i] = new(big.Int).Mod(new(big.Int).Add(v[i], w[i]), EC.N)
    }

    return result
}

func VectorHadamard(v, w []*big.Int) []*big.Int {
    if len(v) != len(w) {
        fmt.Println("VectorHadamard: Uh oh! Arrays not of the same length")
        fmt.Printf("len(v): %d\n", len(w))
        fmt.Printf("len(w): %d\n", len(v))
    }

    result := make([]*big.Int, len(v))

    for i := range v {
        result[i] = new(big.Int).Mod(new(big.Int).Mul(v[i], w[i]), EC.N)
    }

    return result
}

func VectorAddScalar(v []*big.Int, s *big.Int) []*big.Int {
    result := make([]*big.Int, len(v))

    for i := range v {
        result[i] = new(big.Int).Mod(new(big.Int).Add(v[i], s), EC.N)
    }

    return result
}

func ScalarVectorMul(v []*big.Int, s *big.Int) []*big.Int {
    result := make([]*big.Int, len(v))

    for i := range v {
        result[i] = new(big.Int).Mod(new(big.Int).Mul(v[i], s), EC.N)
    }

    return result
}

/*
InnerProd Proof
This stores the argument values
*/
type InnerProdArg struct {
    L []ECPoint
    R []ECPoint
    A *big.Int
    B *big.Int

    Challenges []*big.Int
}

func GenerateNewParams(G, H []ECPoint, x *big.Int, L, R, P ECPoint) ([]ECPoint, []ECPoint, ECPoint) {
    nprime := len(G) / 2

    Gprime := make([]ECPoint, nprime)
    Hprime := make([]ECPoint, nprime)

    xinv := new(big.Int).ModInverse(x, EC.N)

    // Gprime = xinv * G[:nprime] + x*G[nprime:]
    // Hprime = x * H[:nprime] + xinv*H[nprime:]

    for i := range Gprime {
        //fmt.Printf("i: %d && i+nprime: %d\n", i, i+nprime)
        Gprime[i] = G[i].Mult(xinv).Add(G[i+nprime].Mult(x))
        Hprime[i] = H[i].Mult(x).Add(H[i+nprime].Mult(xinv))
    }

    x2 := new(big.Int).Mod(new(big.Int).Mul(x, x), EC.N)
    xinv2 := new(big.Int).ModInverse(x2, EC.N)

    Pprime := L.Mult(x2).Add(P).Add(R.Mult(xinv2)) // x^2 * L + P + xinv^2 * R

    return Gprime, Hprime, Pprime
}

/* Inner Product Argument
Proves that =c
This is a building block for BulletProofs
*/
func InnerProductProveSub(proof InnerProdArg, G, H []ECPoint, a []*big.Int, b []*big.Int, u ECPoint, P ECPoint) InnerProdArg {
    //fmt.Printf("Proof so far: %s\n", proof)
    if len(a) == 1 {
        // Prover sends a & b
        //fmt.Printf("a: %d && b: %d\n", a[0], b[0])
        proof.A = a[0]
        proof.B = b[0]
        return proof
    }

    curIt := int(math.Log2(float64(len(a)))) - 1

    nprime := len(a) / 2
    //fmt.Println(nprime)
    //fmt.Println(len(H))
    cl := InnerProduct(a[:nprime], b[nprime:]) // either this line
    cr := InnerProduct(a[nprime:], b[:nprime]) // or this line
    L := TwoVectorPCommitWithGens(G[nprime:], H[:nprime], a[:nprime], b[nprime:]).Add(u.Mult(cl))
    R := TwoVectorPCommitWithGens(G[:nprime], H[nprime:], a[nprime:], b[:nprime]).Add(u.Mult(cr))

    proof.L[curIt] = L
    proof.R[curIt] = R

    // prover sends L & R and gets a challenge
    s256 := sha256.Sum256([]byte(
        L.X.String() + L.Y.String() +
            R.X.String() + R.Y.String()))

    x := new(big.Int).SetBytes(s256[:])

    proof.Challenges[curIt] = x

    Gprime, Hprime, Pprime := GenerateNewParams(G, H, x, L, R, P)
    //fmt.Printf("Prover - Intermediate Pprime value: %s \n", Pprime)
    xinv := new(big.Int).ModInverse(x, EC.N)

    // or these two lines
    aprime := VectorAdd(
        ScalarVectorMul(a[:nprime], x),
        ScalarVectorMul(a[nprime:], xinv))
    bprime := VectorAdd(
        ScalarVectorMul(b[:nprime], xinv),
        ScalarVectorMul(b[nprime:], x))

    return InnerProductProveSub(proof, Gprime, Hprime, aprime, bprime, u, Pprime)
}

//rpresult.IPP = InnerProductProve(left, right, that, P, EC.U, EC.BPG, HPrime)
func InnerProductProve(a []*big.Int, b []*big.Int, c *big.Int, P, U ECPoint, G, H []ECPoint) InnerProdArg {
    loglen := int(math.Log2(float64(len(a))))

    challenges := make([]*big.Int, loglen+1)
    Lvals := make([]ECPoint, loglen)
    Rvals := make([]ECPoint, loglen)

    runningProof := InnerProdArg{
        Lvals,
        Rvals,
        big.NewInt(0),
        big.NewInt(0),
        challenges}

    // randomly generate an x value from public data
    x := sha256.Sum256([]byte(P.X.String() + P.Y.String()))

    runningProof.Challenges[loglen] = new(big.Int).SetBytes(x[:])

    Pprime := P.Add(U.Mult(new(big.Int).Mul(new(big.Int).SetBytes(x[:]), c)))
    ux := U.Mult(new(big.Int).SetBytes(x[:]))
    //fmt.Printf("Prover Pprime value to run sub off of: %s\n", Pprime)
    return InnerProductProveSub(runningProof, G, H, a, b, ux, Pprime)
}

/* Inner Product Verify
Given a inner product proof, verifies the correctness of the proof
Since we're using the Fiat-Shamir transform, we need to verify all x hash computations,
all g' and h' computations
P : the Pedersen commitment we are verifying is a commitment to the innner product
ipp : the proof
*/

/* Inner Product Verify Fast
Given a inner product proof, verifies the correctness of the proof. Does the same as above except
we replace n separate exponentiations with a single multi-exponentiation.
*/

func InnerProductVerifyFast(c *big.Int, P, U ECPoint, G, H []ECPoint, ipp InnerProdArg) bool {
    //fmt.Println("Verifying Inner Product Argument")
    //fmt.Printf("Commitment Value: %s \n", P)
    s1 := sha256.Sum256([]byte(P.X.String() + P.Y.String()))
    chal1 := new(big.Int).SetBytes(s1[:])
    ux := U.Mult(chal1)
    curIt := len(ipp.Challenges) - 1

    // check all challenges
    if ipp.Challenges[curIt].Cmp(chal1) != 0 {
        fmt.Println("IPVerify - Initial Challenge Failed")
        return false
    }

    for j := curIt - 1; j >= 0; j-- {
        Lval := ipp.L[j]
        Rval := ipp.R[j]

        // prover sends L & R and gets a challenge
        s256 := sha256.Sum256([]byte(
            Lval.X.String() + Lval.Y.String() +
                Rval.X.String() + Rval.Y.String()))

        chal2 := new(big.Int).SetBytes(s256[:])

        if ipp.Challenges[j].Cmp(chal2) != 0 {
            fmt.Println("IPVerify - Challenge verification failed at index " + strconv.Itoa(j))
            return false
        }
    }
    // begin computing

    curIt -= 1
    Pprime := P.Add(ux.Mult(c)) // line 6 from protocol 1

    tmp1 := EC.Zero()
    for j := curIt; j >= 0; j-- {
        x2 := new(big.Int).Exp(ipp.Challenges[j], big.NewInt(2), EC.N)
        x2i := new(big.Int).ModInverse(x2, EC.N)
        //fmt.Println(tmp1)
        tmp1 = ipp.L[j].Mult(x2).Add(ipp.R[j].Mult(x2i)).Add(tmp1)
        //fmt.Println(tmp1)
    }
    rhs := Pprime.Add(tmp1)
    //rhs=P+c*ux + L*xw+R*x2i ??29

    sScalars := make([]*big.Int, EC.V)
    invsScalars := make([]*big.Int, EC.V)

    for i := 0; i < EC.V; i++ {
        si := big.NewInt(1)
        for j := curIt; j >= 0; j-- {
            // original challenge if the jth bit of i is 1, inverse challenge otherwise
            chal := ipp.Challenges[j]
            if big.NewInt(int64(i)).Bit(j) == 0 {
                chal = new(big.Int).ModInverse(chal, EC.N)
            }
            // fmt.Printf("Challenge raised to value: %d\n", chal)
            si = new(big.Int).Mod(new(big.Int).Mul(si, chal), EC.N)
        }
        //fmt.Printf("Si value: %d\n", si)
        sScalars[i] = si
        invsScalars[i] = new(big.Int).ModInverse(si, EC.N)
    }

    ccalc := new(big.Int).Mod(new(big.Int).Mul(ipp.A, ipp.B), EC.N)
    lhs := TwoVectorPCommitWithGens(G, H, ScalarVectorMul(sScalars, ipp.A), ScalarVectorMul(invsScalars, ipp.B)).Add(ux.Mult(ccalc))

    if !rhs.Equal(lhs) {
        fmt.Println("IPVerify - Final Commitment checking failed")
        fmt.Printf("Final rhs value: %s \n", rhs)
        fmt.Printf("Final lhs value: %s \n", lhs)
        return false
    }

    return true
}

// from here: https://play.golang.org/p/zciRZvD0Gr with a fix
func PadLeft(str, pad string, l int) string {
    strCopy := str
    for len(strCopy) < l {
        strCopy = pad + strCopy
    }

    return strCopy
}

func STRNot(str string) string {
    result := ""

    for _, i := range str {
        if i == '0' {
            result += "1"
        } else {
            result += "0"
        }
    }
    return result
}

func StrToBigIntArray(str string) []*big.Int {
    result := make([]*big.Int, len(str))

    for i := range str {
        t, success := new(big.Int).SetString(string(str[i]), 10)
        if success {
            result[i] = t
        }
    }

    return result
}

func reverse(l []*big.Int) []*big.Int {
    result := make([]*big.Int, len(l))

    for i := range l {
        result[i] = l[len(l)-i-1]
    }

    return result
}

func PowerVector(l int, base *big.Int) []*big.Int {
    result := make([]*big.Int, l)

    for i := 0; i < l; i++ {
        result[i] = new(big.Int).Exp(base, big.NewInt(int64(i)), EC.N)
    }

    return result
}

func RandVector(l int) []*big.Int {
    result := make([]*big.Int, l)

    for i := 0; i < l; i++ {
        x, err := rand.Int(rand.Reader, EC.N)
        check(err)
        result[i] = x
    }

    return result
}

func VectorSum(y []*big.Int) *big.Int {
    result := big.NewInt(0)

    for _, j := range y {
        result = new(big.Int).Mod(new(big.Int).Add(result, j), EC.N)
    }

    return result
}

type RangeProof struct {
    Comm ECPoint
    A    ECPoint
    S    ECPoint
    T1   ECPoint
    T2   ECPoint
    Tau  *big.Int
    Th   *big.Int
    Mu   *big.Int
    IPP  InnerProdArg

    // challenges
    Cy *big.Int
    Cz *big.Int
    Cx *big.Int
}

/*
Delta is a helper function that is used in the range proof
\delta(y, z) = (z-z^2)<1^n, y^n> - z^3<1^n, 2^n>
*/

func Delta(y []*big.Int, z *big.Int) *big.Int {
    result := big.NewInt(0)

    // (z-z^2)<1^n, y^n>
    z2 := new(big.Int).Mod(new(big.Int).Mul(z, z), EC.N)
    t1 := new(big.Int).Mod(new(big.Int).Sub(z, z2), EC.N)
    t2 := new(big.Int).Mod(new(big.Int).Mul(t1, VectorSum(y)), EC.N)

    // z^3<1^n, 2^n>
    z3 := new(big.Int).Mod(new(big.Int).Mul(z2, z), EC.N)
    po2sum := new(big.Int).Sub(new(big.Int).Exp(big.NewInt(2), big.NewInt(int64(EC.V)), EC.N), big.NewInt(1))
    t3 := new(big.Int).Mod(new(big.Int).Mul(z3, po2sum), EC.N)

    result = new(big.Int).Mod(new(big.Int).Sub(t2, t3), EC.N)

    return result
}

// Calculates (aL - z*1^n) + sL*x
func CalculateL(aL, sL []*big.Int, z, x *big.Int) []*big.Int {
    result := make([]*big.Int, len(aL))

    tmp1 := VectorAddScalar(aL, new(big.Int).Neg(z))
    tmp2 := ScalarVectorMul(sL, x)

    result = VectorAdd(tmp1, tmp2)

    return result
}

func CalculateR(aR, sR, y, po2 []*big.Int, z, x *big.Int) []*big.Int {
    if len(aR) != len(sR) || len(aR) != len(y) || len(y) != len(po2) {
        fmt.Println("CalculateR: Uh oh! Arrays not of the same length")
        fmt.Printf("len(aR): %d\n", len(aR))
        fmt.Printf("len(sR): %d\n", len(sR))
        fmt.Printf("len(y): %d\n", len(y))
        fmt.Printf("len(po2): %d\n", len(po2))
    }

    result := make([]*big.Int, len(aR))

    z2 := new(big.Int).Exp(z, big.NewInt(2), EC.N)
    tmp11 := VectorAddScalar(aR, z)
    tmp12 := ScalarVectorMul(sR, x)
    tmp1 := VectorHadamard(y, VectorAdd(tmp11, tmp12))
    tmp2 := ScalarVectorMul(po2, z2)

    result = VectorAdd(tmp1, tmp2)

    return result
}

/*
RPProver : Range Proof Prove
Given a value v, provides a range proof that v is inside 0 to 2^64-1
*/
func RPProve(v *big.Int) RangeProof {

    rpresult := RangeProof{}

    PowerOfTwos := PowerVector(EC.V, big.NewInt(2))

    //???????
    if v.Cmp(big.NewInt(0)) == -1 {
        fmt.Println("Value is below range! Not proving")
        os.Exit(1)
    }

    //???????
    if v.Cmp(new(big.Int).Exp(big.NewInt(2), big.NewInt(int64(EC.V)), EC.N)) == 1 {
        fmt.Println("Value is above range! Not proving.")
        os.Exit(1)

    }

    // ?????
    gamma, err := rand.Int(rand.Reader, EC.N)
    check(err)
    comm := EC.G.Mult(v).Add(EC.H.Mult(gamma))
    rpresult.Comm = comm

    //Comm = v*G + gamma*H

    // break up v into its bitwise representation
    //aL := 0
    //??????
    aL := reverse(StrToBigIntArray(PadLeft(fmt.Sprintf("%b", v), "0", EC.V)))
    //??
    aR := VectorAddScalar(aL, big.NewInt(-1))
    //????35

    alpha, err := rand.Int(rand.Reader, EC.N)
    check(err)

    A := TwoVectorPCommitWithGens(EC.BPG, EC.BPH, aL, aR).Add(EC.H.Mult(alpha))
    rpresult.A = A

    // ?????
    //A=(a*G + b*H) + alpha*H

    sL := RandVector(EC.V)
    sR := RandVector(EC.V)

    rho, err := rand.Int(rand.Reader, EC.N)
    check(err)

    S := TwoVectorPCommitWithGens(EC.BPG, EC.BPH, sL, sR).Add(EC.H.Mult(rho))
    rpresult.S = S

    // ?????
    //S=(a*G + b*H) + rho*H

    chal1s256 := sha256.Sum256([]byte(A.X.String() + A.Y.String()))
    cy := new(big.Int).SetBytes(chal1s256[:])

    rpresult.Cy = cy
    //????
    //Cy = sha256(A)

    chal2s256 := sha256.Sum256([]byte(S.X.String() + S.Y.String()))
    cz := new(big.Int).SetBytes(chal2s256[:])

    rpresult.Cz = cz
    //????
    //Cz = sha256(S)
    z2 := new(big.Int).Exp(cz, big.NewInt(2), EC.N)
    // need to generate l(X), r(X), and t(X)=

    /*
        Java code on how to calculate t1 and t2
            FieldVector ys = FieldVector.from(VectorX.iterate(n, BigInteger.ONE, y::multiply),q); //powers of y
            FieldVector l0 = aL.add(z.negate());
            FieldVector l1 = sL;
            FieldVector twoTimesZSquared = twos.times(zSquared);
            FieldVector r0 = ys.hadamard(aR.add(z)).add(twoTimesZSquared);
            FieldVector r1 = sR.hadamard(ys);
            BigInteger k = ys.sum().multiply(z.subtract(zSquared)).subtract(zCubed.shiftLeft(n).subtract(zCubed));
            BigInteger t0 = k.add(zSquared.multiply(number));
            BigInteger t1 = l1.innerPoduct(r0).add(l0.innerPoduct(r1));
            BigInteger t2 = l1.innerPoduct(r1);
            PolyCommitment polyCommitment = PolyCommitment.from(base, t0, VectorX.of(t1, t2));
    */
    PowerOfCY := PowerVector(EC.V, cy)
    // fmt.Println(PowerOfCY)
    l0 := VectorAddScalar(aL, new(big.Int).Neg(cz))
    // l1 := sL
    r0 := VectorAdd(
        VectorHadamard(
            PowerOfCY,
            VectorAddScalar(aR, cz)),
        ScalarVectorMul(
            PowerOfTwos,
            z2))
    r1 := VectorHadamard(sR, PowerOfCY)

    //calculate t0
    t0 := new(big.Int).Mod(new(big.Int).Add(new(big.Int).Mul(v, z2), Delta(PowerOfCY, cz)), EC.N)

    t1 := new(big.Int).Mod(new(big.Int).Add(InnerProduct(sL, r0), InnerProduct(l0, r1)), EC.N)
    t2 := InnerProduct(sL, r1)

    // given the t_i values, we can generate commitments to them
    tau1, err := rand.Int(rand.Reader, EC.N)
    check(err)
    tau2, err := rand.Int(rand.Reader, EC.N)
    check(err)

    T1 := EC.G.Mult(t1).Add(EC.H.Mult(tau1)) //commitment to t1
    T2 := EC.G.Mult(t2).Add(EC.H.Mult(tau2)) //commitment to t2

    rpresult.T1 = T1
    //?????
    //T1 = t1*G + tau1*H
    rpresult.T2 = T2
    //?????
    //T2 = t2*G + tau2*H

    chal3s256 := sha256.Sum256([]byte(T1.X.String() + T1.Y.String() + T2.X.String() + T2.Y.String()))
    cx := new(big.Int).SetBytes(chal3s256[:])

    rpresult.Cx = cx
    //Cx = sha256(T1 + T2)

    left := CalculateL(aL, sL, cz, cx)
    right := CalculateR(aR, sR, PowerOfCY, PowerOfTwos, cz, cx)

    //???????,??????
    thatPrime := new(big.Int).Mod( // t(x) = t0 + t1*x + t2*x^2
        new(big.Int).Add(
            t0,
            new(big.Int).Add(
                new(big.Int).Mul(
                    t1, cx),
                new(big.Int).Mul(
                    new(big.Int).Mul(cx, cx),
                    t2))), EC.N)

    //????? t(x) = 
    that := InnerProduct(left, right) // NOTE: BP Java implementation calculates this from the t_i

    // = t0 + t1*x + t2*x^2
    // thatPrime and that should be equal
    if thatPrime.Cmp(that) != 0 {
        fmt.Println("Proving -- Uh oh! Two diff ways to compute same value not working")
        fmt.Printf("\tthatPrime = %s\n", thatPrime.String())
        fmt.Printf("\tthat = %s \n", that.String())
    }

    rpresult.Th = thatPrime
    //Th = t0 + t1*x + t2*x^2

    taux1 := new(big.Int).Mod(new(big.Int).Mul(tau2, new(big.Int).Mul(cx, cx)), EC.N)
    //t2*x^2
    taux2 := new(big.Int).Mod(new(big.Int).Mul(tau1, cx), EC.N)
    //t1*x
    taux3 := new(big.Int).Mod(new(big.Int).Mul(z2, gamma), EC.N)
    //z2*gamma
    taux := new(big.Int).Mod(new(big.Int).Add(taux1, new(big.Int).Add(taux2, taux3)), EC.N)
    //taux1+taux2+taux3

    rpresult.Tau = taux
    //Tau = taux1 + taux2 + taux3

    mu := new(big.Int).Mod(new(big.Int).Add(alpha, new(big.Int).Mul(rho, cx)), EC.N)
    rpresult.Mu = mu
    //Mu = alpha + rho*Cx

    HPrime := make([]ECPoint, len(EC.BPH))

    for i := range HPrime {
        HPrime[i] = EC.BPH[i].Mult(new(big.Int).ModInverse(PowerOfCY[i], EC.N))
    }

    // for testing
    tmp1 := EC.Zero()
    zneg := new(big.Int).Mod(new(big.Int).Neg(cz), EC.N)
    for i := range EC.BPG {
        tmp1 = tmp1.Add(EC.BPG[i].Mult(zneg))
    }

    tmp2 := EC.Zero()
    for i := range HPrime {
        val1 := new(big.Int).Mul(cz, PowerOfCY[i])
        val2 := new(big.Int).Mul(new(big.Int).Mul(cz, cz), PowerOfTwos[i])
        tmp2 = tmp2.Add(HPrime[i].Mult(new(big.Int).Add(val1, val2)))
    }

    //P1 := A.Add(S.Mult(cx)).Add(tmp1).Add(tmp2).Add(EC.U.Mult(that)).Add(EC.H.Mult(mu).Neg())

    P := TwoVectorPCommitWithGens(EC.BPG, HPrime, left, right)
    //fmt.Println(P1)
    //fmt.Println(P2)

    rpresult.IPP = InnerProductProve(left, right, that, P, EC.U, EC.BPG, HPrime)


    return rpresult
}

func RPVerify(rp RangeProof) bool {
    // verify the challenges
    chal1s256 := sha256.Sum256([]byte(rp.A.X.String() + rp.A.Y.String()))
    cy := new(big.Int).SetBytes(chal1s256[:])
    if cy.Cmp(rp.Cy) != 0 {
        fmt.Println("RPVerify - Challenge Cy failing!")
        return false
    }
    chal2s256 := sha256.Sum256([]byte(rp.S.X.String() + rp.S.Y.String()))
    cz := new(big.Int).SetBytes(chal2s256[:])
    if cz.Cmp(rp.Cz) != 0 {
        fmt.Println("RPVerify - Challenge Cz failing!")
        return false
    }
    chal3s256 := sha256.Sum256([]byte(rp.T1.X.String() + rp.T1.Y.String() + rp.T2.X.String() + rp.T2.Y.String()))
    cx := new(big.Int).SetBytes(chal3s256[:])
    if cx.Cmp(rp.Cx) != 0 {
        fmt.Println("RPVerify - Challenge Cx failing!")
        return false
    }

    // given challenges are correct, very range proof
    PowersOfY := PowerVector(EC.V, cy)

    // t_hat * G + tau * H
    lhs := EC.G.Mult(rp.Th).Add(EC.H.Mult(rp.Tau))

    // z^2 * V + delta(y,z) * G + x * T1 + x^2 * T2
    rhs := rp.Comm.Mult(new(big.Int).Mul(cz, cz)).Add(
        EC.G.Mult(Delta(PowersOfY, cz))).Add(
        rp.T1.Mult(cx)).Add(
        rp.T2.Mult(new(big.Int).Mul(cx, cx)))

    if !lhs.Equal(rhs) {
        fmt.Println("RPVerify - Uh oh! Check line (63) of verification")
        fmt.Println(rhs)
        fmt.Println(lhs)
        return false
    }

    tmp1 := EC.Zero()
    zneg := new(big.Int).Mod(new(big.Int).Neg(cz), EC.N)
    for i := range EC.BPG {
        tmp1 = tmp1.Add(EC.BPG[i].Mult(zneg))
    }
    //tmp1+=BPG*(-cz)

    PowerOfTwos := PowerVector(EC.V, big.NewInt(2))
    tmp2 := EC.Zero()
    // generate h'
    HPrime := make([]ECPoint, len(EC.BPH))

    for i := range HPrime {
        mi := new(big.Int).ModInverse(PowersOfY[i], EC.N)
        HPrime[i] = EC.BPH[i].Mult(mi)
    }

    for i := range HPrime {
        val1 := new(big.Int).Mul(cz, PowersOfY[i])
        val2 := new(big.Int).Mul(new(big.Int).Mul(cz, cz), PowerOfTwos[i])
        tmp2 = tmp2.Add(HPrime[i].Mult(new(big.Int).Add(val1, val2)))
    }

    //tmp2+=BPH*(val1+val2)

    // without subtracting this value should equal muCH + l[i]G[i] + r[i]H'[i]
    // we want to make sure that the innerproduct checks out, so we subtract it
    P := rp.A.Add(rp.S.Mult(cx)).Add(tmp1).Add(tmp2).Add(EC.H.Mult(rp.Mu).Neg())
    //fmt.Println(P)
    //P=A+cx*S+tmp1+tmp2+(-Mu*H)

    if !InnerProductVerifyFast(rp.Th, P, EC.U, EC.BPG, HPrime, rp.IPP) {
        fmt.Println("RPVerify - Uh oh! Check line (65) of verification!")
        return false
    }

    return true
}

// Calculates (aL - z*1^n) + sL*x
func CalculateLMRP(aL, sL []*big.Int, z, x *big.Int) []*big.Int {
    result := make([]*big.Int, len(aL))

    tmp1 := VectorAddScalar(aL, new(big.Int).Neg(z))
    tmp2 := ScalarVectorMul(sL, x)

    result = VectorAdd(tmp1, tmp2)

    return result
}

func CalculateRMRP(aR, sR, y, zTimesTwo []*big.Int, z, x *big.Int) []*big.Int {
    if len(aR) != len(sR) || len(aR) != len(y) || len(y) != len(zTimesTwo) {
        fmt.Println("CalculateR: Uh oh! Arrays not of the same length")
        fmt.Printf("len(aR): %d\n", len(aR))
        fmt.Printf("len(sR): %d\n", len(sR))
        fmt.Printf("len(y): %d\n", len(y))
        fmt.Printf("len(po2): %d\n", len(zTimesTwo))
    }

    result := make([]*big.Int, len(aR))

    tmp11 := VectorAddScalar(aR, z)
    tmp12 := ScalarVectorMul(sR, x)
    tmp1 := VectorHadamard(y, VectorAdd(tmp11, tmp12))

    result = VectorAdd(tmp1, zTimesTwo)

    return result
}

/*
DeltaMRP is a helper function that is used in the multi range proof
\delta(y, z) = (z-z^2)<1^n, y^n> - \sum_j z^3+j<1^n, 2^n>
*/

func DeltaMRP(y []*big.Int, z *big.Int, m int) *big.Int {
    result := big.NewInt(0)

    // (z-z^2)<1^n, y^n>
    z2 := new(big.Int).Mod(new(big.Int).Mul(z, z), EC.N)
    t1 := new(big.Int).Mod(new(big.Int).Sub(z, z2), EC.N)
    t2 := new(big.Int).Mod(new(big.Int).Mul(t1, VectorSum(y)), EC.N)

    // \sum_j z^3+j<1^n, 2^n>
    // <1^n, 2^n> = 2^n - 1
    po2sum := new(big.Int).Sub(new(big.Int).Exp(big.NewInt(2), big.NewInt(int64(EC.V/m)), EC.N), big.NewInt(1))
    t3 := big.NewInt(0)

    for j := 0; j < m; j++ {
        zp := new(big.Int).Exp(z, big.NewInt(3+int64(j)), EC.N)
        tmp1 := new(big.Int).Mod(new(big.Int).Mul(zp, po2sum), EC.N)
        t3 = new(big.Int).Mod(new(big.Int).Add(t3, tmp1), EC.N)
    }

    result = new(big.Int).Mod(new(big.Int).Sub(t2, t3), EC.N)

    return result
}

// NewECPrimeGroupKey returns the curve (field),
// Generator 1 x&y, Generator 2 x&y, order of the generators
func NewECPrimeGroupKey(n int) CryptoParams {
    curValue := btcec.S256().Gx
    s256 := sha256.New()
    gen1Vals := make([]ECPoint, n)
    gen2Vals := make([]ECPoint, n)
    u := ECPoint{big.NewInt(0), big.NewInt(0)}
    cg := ECPoint{}
    ch := ECPoint{}

    j := 0
    confirmed := 0
    for confirmed < (2*n + 3) {
        s256.Write(new(big.Int).Add(curValue, big.NewInt(int64(j))).Bytes())

        potentialXValue := make([]byte, 33)
        binary.LittleEndian.PutUint32(potentialXValue, 2)
        for i, elem := range s256.Sum(nil) {
            potentialXValue[i+1] = elem
        }

        gen2, err := btcec.ParsePubKey(potentialXValue, btcec.S256())
        if err == nil {
            if confirmed == 2*n { // once we've generated all g and h values then assign this to u
                u = ECPoint{gen2.X, gen2.Y}
                //fmt.Println("Got that U value")
            } else if confirmed == 2*n+1 {
                cg = ECPoint{gen2.X, gen2.Y}

            } else if confirmed == 2*n+2 {
                ch = ECPoint{gen2.X, gen2.Y}
            } else {
                if confirmed%2 == 0 {
                    gen1Vals[confirmed/2] = ECPoint{gen2.X, gen2.Y}
                    //fmt.Println("new G Value")
                } else {
                    gen2Vals[confirmed/2] = ECPoint{gen2.X, gen2.Y}
                    //fmt.Println("new H value")
                }
            }
            confirmed += 1
        }
        j += 1
    }

    return CryptoParams{
        btcec.S256(),
        btcec.S256(),
        gen1Vals,
        gen2Vals,
        btcec.S256().N,
        u,
        n,
        cg,
        ch}
}

func init() {
    EC = NewECPrimeGroupKey(VecLength)

}
func main() {
    argCount := len(os.Args[1:])
    val:="1"

        if (argCount>0) { val = string(os.Args[1]) }
    if (argCount>1) { VecLength,_  = strconv.Atoi((os.Args[2])) }

    EC = NewECPrimeGroupKey(VecLength)
    m,_ := new(big.Int).SetString(val, 10)
    rtn:=RPProve(m) 
    r:=RPVerify(rtn)
    fmt.Printf("Value is : %s\n",val)
    fmt.Printf("Value is between 1 and 2^%d-1: %t\n",VecLength,r)

    
    fmt.Printf("=== Public parameters:\n")
    fmt.Printf(" Curve type:\tsecp256k1\n")
    fmt.Printf(" G:\t%s\n",EC.G)
    fmt.Printf(" H:\t%s\n",EC.H)
    fmt.Printf(" Curve b value:\t%s\n",EC.C.Params().B)
    fmt.Printf(" Curve prime value:\t%s\n",EC.C.Params().P)
    fmt.Printf(" Gi[0]:\t%s\n",EC.BPG[0])
    fmt.Printf(" Hi[0]:\t%s\n",EC.BPH[0])
    fmt.Printf(" Vector length:\t%d\n",EC.V)



    fmt.Printf("\n=== Proof\n")
    fmt.Printf("Challenge:\n")
    fmt.Printf(" Cx:\t%s\n",rtn.Cx)
    fmt.Printf(" Cy:\t%s\n",rtn.Cy)
    fmt.Printf(" Cz:\t%s\n",rtn.Cz)
    fmt.Printf("A:\t%s\n",rtn.A)
    fmt.Printf("S:\t%s\n",rtn.S)
    fmt.Printf("T1:\t%s\n",rtn.T1)
    fmt.Printf("T2:\t%s\n",rtn.T2)
    fmt.Printf("Tau:\t%s\n",rtn.Tau)
    fmt.Printf("Th:\t%s\n",rtn.Th)
    fmt.Printf("Mu:\t%s\n",rtn.Mu)

    fmt.Printf("\nIPP (Inner product proof):\n")
    fmt.Printf(" a:\t%s\n",rtn.IPP.A)
    fmt.Printf(" b:\t%s\n",rtn.IPP.B)
    fmt.Printf(" L[0]:\t%s\n",rtn.IPP.L[0])
    fmt.Printf(" R[0]:\t%s\n",rtn.IPP.R[0])
    fmt.Printf(" L[1]:\t%s\n",rtn.IPP.L[1])
    fmt.Printf(" R[1]:\t%s\n",rtn.IPP.R[1])


}


In [ ]:
Value is : 255
Value is between 1 and 2^8-1: true
=== Public parameters:
 Curve type:    secp256k1
 G: {19049299976918701640434378399416395322521814324735570823046957748159542780110 32970026245044992563886736445720684210429064727746262888414787291191552501274}
 H: {65902674721981107959526451160629142373497501987645733246213038919069206754027 11211914402750177729239671736547089198200305441898725665542549987877079109934}
 Curve b value: 7
 Curve prime value: 115792089237316195423570985008687907853269984665640564039457584007908834671663
 Gi[0]: {72242507303365076728861865557797731214902786606643414733580150814770869272593 27033601650718423694844620558628339447280369833685083662535239836251867410934}
 Hi[0]: {5585562941503279596474839515994484383064083005553472397059910728304658620745 77310285201950507048734835505325678526225389174038208505235173650760230890492}
 Vector length: 8

=== Proof
Challenge:
 Cx:    66873176456638195897075093943820216817246979642048211473051711611923467228895
 Cy:    94074857772546105393432567127684826293641863216229224503445427122548233017721
 Cz:    91671346013715555422665248380246271672435719609721904941373147691938060218262
A:  {99151663132386421360826436734170381859123841654083969527633399460065061486436 111482165979079479057212408659821512649015060474221455405197742567550385897115}
S:  {24387856947604846374579509788424934668318630868248825224444381198936741547943 91597811734661403355672824800944939146866056684783543174275364115189273660156}
T1: {50701721388410539432219502110285267682852029218476737003678322320819136719789 100165105171911445858733626563137723680935104359796425756337939880413561447551}
T2: {790606333868662503386096381303248430313160991250374129788864023291738481436 76988700860812809559385401453275912208647775368392255937654377330591920379597}
Tau:    82672454385139165066116662283889751136007676169056119223649069916753949048794
Th: 51521439334729587135285739526658412104300083857778012182400339436050545359344
Mu: 107321052306978628303630979996854527268548846141216344664377080735838067344915

IPP (Inner product proof):
 a: 108507359215445469625362809755510233879009465046384376071302252483567473805857
 b: 55704077063196037722414059103854613624677335973528507264544408761416602693623
 L[0]:  {114512677526766969335542630484381031751539257942891280777452927194288458720950 51791897630964214328227726952034912012594454551218173333768556123862561210745}
 R[0]:  {64572451821403647963119237130713972117313225054906284814354108403842082428724 19767350298049787570483059248801228110335064021067596332350582196982352199498}
 L[1]:  {75306967628183584964594940304598890001841334825877625745872473027502537372848 88539102357437846679263632799148301286486928220698346592933528481079305656409}
 R[1]:  {91586174703118349305711798800544605175603206581502496629809610785446690258254 46512665635955618974229266885074811509645919685052613413520612491966763789825}

In [ ]:
/*###############################################################################################*/
/*################################  Prover    ##################################################*/
/*###############################################################################################*/


const util = require('util');


function rangeBpProver(x1,pedCom1,r0,r1){

  const crypto = require('crypto');
  const BigInteger = require('big-integer');
  const Consts = require('./consts');
  const utils = require('./utils');
  const pickRandom = utils.pickRandom;
  const modulo = utils.modulo;
  const moduloPow = utils.moduloPow;
  const moduloAddq = utils.moduloAddq;
  const moduloSubq = utils.moduloSubq;
  const moduloMulq = utils.moduloMulq;
  const moduloMul = utils.moduloMul;



  const aL = [];
  const aR = [];
  const SL = [];
  const SR = [];

  const alpha = pickRandom(Consts.q);
  const rho = pickRandom(Consts.q);

  const gVector = [];
  const hVector = []; 
  var gRand;
  var hRand;
  const H = utils.ec.g.mul((r0.toString(Consts.HEX)));
  let v = 0;
  while(v< Consts.upperBoundNumBits){
    gRand = utils.ec.g.x.fromRed().toString(16).concat(BigInteger(v).toString(Consts.HEX));
    hRand = H.x.fromRed().toString(16).concat(BigInteger(v).toString(Consts.HEX));
    gVector[v] = utils.ec.g.mul(modulo(BigInteger(crypto.createHash('sha256').update(gRand).digest('hex'),Consts.HEX),Consts.q).toString(Consts.HEX));
    hVector[v] = H.mul(modulo(BigInteger(crypto.createHash('sha256').update(hRand).digest('hex'),Consts.HEX),Consts.q).toString(Consts.HEX));
    v++;
  }


  var A  = H.mul(alpha.toString(Consts.HEX));
  var S = H.mul(rho.toString(Consts.HEX));
  let i = 0;
  while(i< Consts.upperBoundNumBits){
    aL[i]=x1.shiftRight(i).mod(2).and(BigInteger(1));
    aR[i]= moduloSubq(aL[i],BigInteger(1));
    A = A.add(gVector[i].mul(aL[i].toString(Consts.HEX))).add(hVector[i].mul(aR[i].toString(Consts.HEX)));
    //A = A.add(Q.mul(aL[i].toString(Consts.HEX))).add(Q.mul(aR[i].toString(Consts.HEX)));
    SL[i] = pickRandom(Consts.q);
    SR[i] = pickRandom(Consts.q);
    S = (S).add(gVector[i].mul(SL[i].toString(Consts.HEX))).add(hVector[i].mul(SR[i].toString(Consts.HEX)));
    //S = (S).add(Q.mul(SL[i].toString(Consts.HEX))).add(Q.mul(SR[i].toString(Consts.HEX)));
    i++;
  }


  const y_str = crypto.createHash('sha256').update(A.x.fromRed().toString(16).concat(S.x.fromRed().toString(16))).digest('hex');
  const z_str = crypto.createHash('sha256').update(A.x.fromRed().toString(16).concat(S.x.fromRed().toString(16).concat(y_str))).digest('hex');
  const y = BigInteger(y_str,Consts.HEX);
  const z = BigInteger(z_str,Consts.HEX);


  const zSquared = moduloPow(z,2,Consts.q);
  const zCubed = moduloPow(z,3,Consts.q);
  var t0  = modulo(modulo(zSquared,Consts.q).multiply(modulo(x1,Consts.q)),Consts.q);
  var t0Part1;
  var t0Part2;
  var t0Part3;
  var t1 = BigInteger(0);
  var t1Part1;
  var t1Part2;
  var t1Part3;
  var t1Part4;
  var t1Part5;
  var t1Part6;
  var t1Part7;
  var t1Part8;
  var t2 = BigInteger(0);
  var t2Part1;
  var t2Part2;
  var t2Part3;
  var yi = [];
  let s = 0;
  while(s< Consts.upperBoundNumBits){
    yi[s] = moduloPow(y,s,Consts.q);
    t0Part1 = modulo(modulo(z,Consts.q).multiply(modulo(yi[s],Consts.q)),Consts.q);
    t0Part2 = modulo(modulo(zSquared,Consts.q).multiply(modulo(yi[s],Consts.q)),Consts.q);
    t0Part3 = modulo(modulo(zCubed,Consts.q).multiply(moduloPow(BigInteger(2),s,Consts.q)),Consts.q);
    t0 = moduloSubq(moduloSubq(moduloAddq(t0,t0Part1),t0Part2),t0Part3);
    //t0 = modulo(modulo(modulo(t0.add(t0Part1),Consts.q).subtract(t0Part2),Consts.q).subtract(t0Part3),Consts.q);
    //t0 = t0.add(z.multiply(yi[i]).add(zSquared.multiply(yi[i]).multiply(-1)).add(zCubed.multiply(BigInteger(2).pow(i)).multiply(-1)));
    t1Part1 = modulo(aR[s].add(z),Consts.q);
    t1Part2 = modulo(modulo(t1Part1,Consts.q).multiply(modulo(yi[s],Consts.q)),Consts.q);
    t1Part3 = modulo(modulo(SL[s],Consts.q).multiply(modulo(t1Part2,Consts.q)),Consts.q);
    t1Part4 = moduloSubq(aL[s],z);
    //t1Part4 = modulo(aL[i].subtract(z),Consts.q);
    t1Part5 = modulo(modulo(SR[s],Consts.q).multiply(modulo(yi[s],Consts.q)),Consts.q);
    t1Part6 = modulo(modulo(t1Part4,Consts.q).multiply(modulo(t1Part5,Consts.q)),Consts.q);
    t1Part7 = modulo(modulo(zSquared,Consts.q).multiply(moduloPow(BigInteger(2),s,Consts.q)),Consts.q);
    t1Part8 = moduloMulq(t1Part7,SL[s]);
    t1 = moduloAddq(moduloAddq(moduloAddq(t1,t1Part3),t1Part6),t1Part8);
   // t1 = modulo(modulo(t1.add(t1Part3),Consts.q).add(t1Part6),Consts.q);
    t2Part1 = modulo(modulo(SR[s],Consts.q).multiply(modulo(yi[s],Consts.q)),Consts.q);
    t2Part2 = modulo(modulo(SL[s],Consts.q).multiply(modulo(t2Part1,Consts.q)),Consts.q);
    t2 = modulo(t2.add(t2Part2),Consts.q);
    //t1 = t1.add((SL[i].multiply((aR[i].add(z)).multiply(yi[i]))).add((aL[i].subtract(z)).multiply(SR[i].add(yi[i]))));
    //t2 = t2.add(SL[i].multiply(SR[i].multiply(yi[i])));
    s++;
  }
  const tau1 = pickRandom(Consts.q);
  const tau2 = pickRandom(Consts.q);
  const T1 = utils.ec.g.mul(t1.toString(Consts.HEX)).add(H.mul(tau1.toString(Consts.HEX)));
  const T2 = utils.ec.g.mul(t2.toString(Consts.HEX)).add(H.mul(tau2.toString(Consts.HEX)));

  //fiat shamir for verifier challenge: 
  const concatStrings = T1.x.fromRed().toString(16).concat(T2.x.fromRed().toString(16)).concat(H.x.fromRed().toString(16));
  const temp = crypto.createHash('sha256').update(concatStrings).digest('hex');
  const xFiatShamirChall = modulo(BigInteger(temp,Consts.HEX),Consts.q);
  const xFiatShamirChallSquared = moduloPow(xFiatShamirChall, 2,Consts.q);
  //(A * B) mod C = (A mod C * B mod C) mod C
  const tauPart1 = modulo(modulo(tau1,Consts.q).multiply(modulo(xFiatShamirChall,Consts.q)),Consts.q);
  const tauPart2 = modulo(modulo(tau2,Consts.q).multiply(xFiatShamirChallSquared),Consts.q);
  const tauPart3 = modulo(modulo(zSquared,Consts.q).multiply(modulo(r1,Consts.q)),Consts.q);
  const tauX = modulo(modulo(tauPart1.add(tauPart2),Consts.q).add(tauPart3),Consts.q);
  const miuPart1 = modulo(modulo(rho,Consts.q).multiply(modulo(xFiatShamirChall,Consts.q)),Consts.q);
  const miu =  modulo(alpha.add(miuPart1),Consts.q);
  var Lp = [];
  var LpPart1;
  var Rp = [];
  var RpPart1;
  var RpPart2;
  var RpPart3;
  var RpPart4;
  var tX = BigInteger(0);
  var tXPart1;
  var j = 0;
  while(j< Consts.upperBoundNumBits){
    //(A + B) mod C = (A mod C + B mod C) mod C
    LpPart1 = modulo(modulo(SL[j],Consts.q).multiply(modulo(xFiatShamirChall,Consts.q)),Consts.q);
    //Lp[j] = modulo(modulo(aL[j].subtract(z),Consts.q).add(LpPart1),Consts.q);
    Lp[j] = moduloAddq(moduloSubq(aL[j],z),LpPart1);    
    //Lp[j] = aL[j].subtract(z).add(SL[j].multiply(xFiatShamirChall));
    RpPart1 = modulo(modulo(SR[j],Consts.q).multiply(modulo(xFiatShamirChall,Consts.q)),Consts.q);
    RpPart2 = modulo(modulo(zSquared,Consts.q).multiply(moduloPow(BigInteger(2),j,Consts.q)),Consts.q);
    RpPart3 = modulo(modulo(aR[j].add(z),Consts.q).add(RpPart1),Consts.q);
    RpPart4 = modulo(modulo(yi[j],Consts.q).multiply(modulo(RpPart3,Consts.q)),Consts.q);
    Rp[j] = modulo(RpPart4.add(RpPart2),Consts.q);
    //Rp[j] = (yi[j].multiply(aR[j].add(z).add(SR[j].multiply(xFiatShamirChall)))).add(zSquared.multiply(BigInteger(2).pow(j)));
    tXPart1 = modulo(modulo(Lp[j],Consts.q).multiply(modulo(Rp[j],Consts.q)),Consts.q);
    tX = modulo(tX.add(tXPart1),Consts.q);
    //tX = tX.add(Lp[j].multiply(Rp[j]));
    j++;
  }

  const transcript1 = tauX.toString(Consts.HEX).concat(miu.toString(Consts.HEX)).concat(tX.toString(Consts.HEX));
  const NIchallenge1 = crypto.createHash('sha256').update(transcript1).digest('hex');
  const nic1 = modulo(BigInteger(NIchallenge1,Consts.HEX),Consts.q);


  let k=0;

  var P = utils.ec.g.mul(nic1.toString(Consts.HEX)).mul(tX.toString(Consts.HEX));
  //var P = utils.ec.g.mul(0);

  var hiTag = [];
  var yiInv = [];
  while(k=1){
   L[j]= utils.ec.g.mul('0'); //init
   R[j]=utils.ec.g.mul('0');  //init
    cL=BigInteger(0);
    cR=BigInteger(0);
    i1=0;

    while (i1
  var t0 = BigInteger(0);
  var t0Part1;
  var t0Part2;
  var t0Part3;
  let j = 0;
  while(j< Consts.upperBoundNumBits){
    t0Part1 = modulo(modulo(z,Consts.q).multiply(modulo(yi[j],Consts.q)),Consts.q);
    t0Part2 = modulo(modulo(zSquared,Consts.q).multiply(modulo(yi[j],Consts.q)),Consts.q);
    t0Part3 = modulo(modulo(zCubed,Consts.q).multiply(moduloPow(BigInteger(2),j,Consts.q)),Consts.q);
    //t0 = modulo(modulo(modulo(t0.add(t0Part1),Consts.q).subtract(t0Part2),Consts.q).subtract(t0Part3),Consts.q);
    t0 = moduloSubq(moduloSubq(moduloAddq(t0,t0Part1),t0Part2),t0Part3);
    j++;
  }

  // fiat shamir challenge  line 50
  const concatStrings = T1.x.fromRed().toString(16).concat(T2.x.fromRed().toString(16)).concat(H.x.fromRed().toString(16));
  const temp = crypto.createHash('sha256').update(concatStrings).digest('hex');
  const xFiatShamirChall = modulo(BigInteger(temp,Consts.HEX),Consts.q);
  const xFiatShamirChallSquared = moduloPow(xFiatShamirChall,2,Consts.q);
   
  const eq63LeftSide = (utils.ec.g.mul(tX.toString(Consts.HEX))).add(H.mul(tauX.toString(Consts.HEX)));
  const eq63RightSide = (utils.ec.g.mul(t0.toString(Consts.HEX))).add(pedCom1.mul(zSquared.toString(Consts.HEX))).add(T1.mul(xFiatShamirChall.toString(Consts.HEX))).add(T2.mul(xFiatShamirChallSquared.toString(Consts.HEX)));
  if(eq63LeftSide.x.fromRed().toString(16)!=eq63RightSide.x.fromRed().toString(16)){result10=false;}
  if(eq63LeftSide.y.fromRed().toString(16)!=eq63RightSide.y.fromRed().toString(16)){result10=false;}
  //inner product proof:
  // P
  const transcript1 = tauX.toString(Consts.HEX).concat(miu.toString(Consts.HEX)).concat(tX.toString(Consts.HEX)); //33
  const NIchallenge1 = crypto.createHash('sha256').update(transcript1).digest('hex');
  const nic1 = BigInteger(NIchallenge1,Consts.HEX);


  //line 62 :
 
 var P = utils.ec.g.mul(nic1.toString(Consts.HEX)).mul(tX.toString(Consts.HEX)).add(H.mul(((Consts.q).subtract(miu)).toString(Consts.HEX)))

  P = P.add(A).add(S.mul(xFiatShamirChall.toString(Consts.HEX)));


  var hExponent = [];
  let k = 0;
  while(k< Consts.upperBoundNumBits){
    hExponent[k] = moduloAddq(moduloMulq(z,yi[k]),moduloMulq(zSquared,moduloPow(BigInteger(2),k,Consts.q)));
  P = P.add(gVector[k].mul(((Consts.q).subtract(z)).toString(Consts.HEX))).add(hiTag[k].mul(hExponent[k].toString(Consts.HEX)));

    k++;
  }

  var Ptag = P;
  const nPad = Consts.upperBoundNumBits;
  var nTag = nPad/2;
  var i2;

  var transcript;
  var NIchallenge;
  var x;
  var xinv;
  var xSquare;
  var xSquareInv;
  var gVectorTag= gVector;
    var hVectorTag = hiTag;
   j = 0;
  while(nTag>=1){
    
     transcript = L[j].x.fromRed().toString(16).concat(R[j].x.fromRed().toString(16)).concat(H.x.fromRed().toString(16));
     NIchallenge = crypto.createHash('sha256').update(transcript).digest('hex');
      x = BigInteger(NIchallenge,Consts.HEX);
      xinv = x.modInv(Consts.q);
    xSquare = moduloPow(x,2,Consts.q);
    xSquareInv = xSquare.modInv(Consts.q);
    gVector = gVectorTag;
    hiTag = hVectorTag;
     gVectorTag = [];
     hVectorTag = [];
 
  
    i2=0;
    while (i2256){console.log("error: upper bound should be <256bits");return;}

  if(padSize>0){console.log("error: range works only for powers of 2 for now");return;}

 // const x1 = pickRandom(BigInteger(2).pow(Consts.upperBoundNumBits));


var args = process.argv;
if (args.length>1) message=args[2];


  const x1=BigInteger(message);

  const r0 = pickRandom(Consts.q);
  const r1 = pickRandom(Consts.q);
  const pedCom1 = utils.ec.g.mul(x1.toString(Consts.HEX)).add(utils.ec.g.mul((r0.multiply(r1)).toString(Consts.HEX)));

  const {A,S,T1,T2,tauX,miu,tX,L,R,aTag,bTag} = rangeBpProver(x1,pedCom1,r0,r1);


  const result10 = rangeBpVerifier(r0,r1,pedCom1,A,S,T1,T2,tauX,miu,tX,L,R,aTag,bTag);
  if(result10 == false){} //abort

  str="Value:"+message
  str+="\nBits:"+Consts.upperBoundNumBits

str+="\nProof:"+result10
str+="\nR0: "+r0
str+="\nR1: "+r1
str+="\nPed com: "+pedCom1.x+", "+pedCom1.y


str+=util.format("\n\nChallenge")
str+=util.format("\nA: %s, %s",A.x,A.y)
str+=util.format("\nS: %s, %s",S.x,S.y)

str+=util.format("\nT1: %s, %s",T1.x,T1.y)
str+=util.format("\nT2: %s, %s",T2.x,T2.y)
str+=util.format("\nTaux: %s",tauX)
str+=util.format("\nMiu: %s",miu)
str+=util.format("\ntX: %s",tX)

str+=util.format("\n\nIPP (Inner product proof)")
str+=util.format("\naTag: %s",aTag)
str+=util.format("\nbTag: %s",bTag)

str+=util.format("\nL[0]: %s, %s",L[0].x,L[0].y)
str+=util.format("\nR[0]: %s, %s",R[0].x,R[0].y)
str+=util.format("\nL[1]: %s, %s",L[1].x,L[1].y)
str+=util.format("\nR[1]: %s, %s",R[1].x,R[1].y)


str=str.replace("<","lt;")
str=str.replace(">","gt;")
console.log(str)


}


controller()



module.exports = {
  controller,
};

In [ ]:
Value:128
Bits:4
Proof:false
R0: 1.1824692793311763e+76
R1: 4.240133259900473e+76
Ped com: 68119130198504538834379955525603971000788069048233759713490825825127164003644, 51220078235203101089612418110015071369556315242440243843324708243413012612998

Challenge
A: 19110614949710245228550808498917172492581441670020642335880570542435616991171, 55181656154658274394879974072341835840269121084776816307418531831181614870091
S: 38947793637093928990177204447428440964382703980081341002880063642615033973581, 77300232897177946139059592400965522391932419817312998429121737527765154039432
T1: 26045048044069343013032983046593652559227436982880673388370675918032934382434, 76207401378654274186219132229361043019705452538065285629759988631157399033452
T2: 109468214682091997854008174863163211059590584892653822904095070378332519331954, 86640221792101495033916998160947918274072598882693982709290319697020996801637
Taux: 21119936003448097665272896760812432647754321931748082711088720156058040909216
Miu: 97822483036319224835438138323726249524909899770368623230486794623484895043049
tX: 5019271808078520249764680465700420187244125916152676035299748521794749963872

IPP (Inner product proof)
aTag: 17087222763525367134095279024575454796683881316217889813554465309658883096891
bTag: 52320303634529228557255981988194311411535018420318841517830916203824383831286
L[0]: 870247247096461317857281204741727870809401464519486453675744641503998033798, 74867421472366630988957423998594298310814602352853526219171288546210722596769
R[0]: 73699541622994332878408518052140832496122474448911758437576915353942946358965, 83042006817917134264618523365719861831989081804473271489580165742231885431392
L[1]: 106839663194368298929869973662228289115178670942910164077507081098607436710035, 87250207070619012527586431184627825171593886630974351027776838892470678015169
R[1]: 80647853534663966114520015568460799783557110565369836632642133131311559595699, 27942902794302065429418606801482870587705435059683822216637334447319326024563

# Damgard-Fujisaki zero knowledge proof

[Source](https://link.springer.com/content/pdf/10.1007/3-540-36178-2_8.pdf)

## DF ZKP of knowing secret $x$

Alice will prove she knows $x$, without revealing $x$.

In [5]:
from Crypto.Util import number

* Alice and Bob agree on two bases for their calculations $(g,h)$ and a prime $(n)$
* Bob sends Alice a random number, $r$
* Alice generates her own random number, $r_1$

$
c_1 = g^x h^{r_1} \\
r_2 = r - xr_1 \\
c_2 = c_1^x h^{r_2} \mod n \\
$

Alice sends Bob $c_2$. Bob tests if $c_2 == g ^{x^2} h^r \mod n$. The above works because:

* If Alice does not have a specific number in mind, she cannot generate both $g^x$ and $xr_1$.
* Alice hides $x$ with $r_1$.
* For Bob to find $g^x$ he must solve discrete log prob.
* Note that $c_1^x h^{r_2} \mod n = (g^x h^{r_1})^x h^{r - xr_1} \mod n$ 
* Which gives $g^{x^2} h^{xr_1} \cdot h^{r - xr_1} \mod n = g^{x^2}h^r \mod n$

In [6]:
import libnum
import random

In [7]:
def c2(c_1, x, h, r_2, n):
    if r_2 > 0:
        return pow(c_1, x, n) * libnum.invmod(pow(h, -r_2, n), n) % n
    else:
        return pow(c_1, x, n) * pow(h, r_2, n) % n

In [8]:
# We first agree on generators and modulus
g,h,n = 3,5,pow(2,19)-1

In [9]:
# Alice picks a number x, gets a random r from Bob, generates her own random r_1
x = 2
r = random.getrandbits(16)
r_1 = random.getrandbits(16)

# Alice computes c_1 and c_2
c_1 = pow(g, x, n) * pow(h, r_1, n) % n
r_2 = (r - x * r_1)
c_2 = c2(c_1, x, h, r_2, n)

#Alice sends c_2 and g^(x^2)h^r to Bob, who compares
c_2 == pow(g, x * x, n) * pow(h, r, n) % n # true IFF Alice had a number in mind.

True

## DF ZKP method for $x > 0$ proof

Alice has to prove that she has a positive value.

* Actors agree on bases $(g,h)$ and prime $p$.
* Bob sends Alice random challenge $r$
* Every positive value can be [represented with four squares](http://oeis.org/wiki/Sums_of_squares) $x = \{x_0^2 + x_1^2 + x_2^2 + x_3^2 \vert x_i \in \mathbb{I}\}$ 
* Use [Rabin Shallit](https://math.stackexchange.com/questions/483101/rabin-and-shallit-algorithm) or see [source](https://stackoverflow.com/questions/41524508/express-a-given-number-as-a-sum-of-four-squares)
* Alice creates four commitments, on for each $x_i$ as follows
 1. $r_0, r_1, r_2, r_3 \leftarrow \texttt{randInt() for i in range(4)}$
 2. $c_0 = g^{x^2_0} h^{r_0} \mod n$
 3. $c_1 = g^{x^2_1} h^{r_1} \mod n$
 4. $c_2 = g^{x^2_2} h^{r_2} \mod n$
 5. $c_3 = g^{x^2_3} h^{r_3} \mod n$
* Alice commits $r = r_0 + r_1 + r_2 + r_3$
* Bob checks $c = c_0 \times c_1 \times c_2 \times c_3 == g^{x} h^r \mod n$
* If Alice has a negative number, she cannot represent it with four squares
* Bob cannot find $x$ without discrete log.
* Why do you need h?

In [10]:
import numpy as np

In [11]:
# I do not know why you would need h here. Probably to provide added security for x but I am not sure.
# To simplify I omit the h part as it is trivially easy to add.

# Alice and Bob agree on bases g,h and prime n
g,h,n = 3,5,pow(2,19)-1

# # Bob sends over r to Alice
# r = random.getrandbits(16)

# # Alice generates 4 random values that sum to r
# r_0 = r - random.randint(r // 4, 3 * r // 4)
# r_1 = r_0 - random.randint(r_0 // 4, 3 * r_0 // 4)
# r_2 = r_1 - random.randint(r_1 // 4, 3 * r_1 // 4)
# r_3 = r - r_0 - r_1 - r_2

# Alice number, normally you would find the elements but this is easier
x = [random.getrandbits(8) ** 2 for i in range(4)]
x_sum = int(np.sum(x))
c_0, c_1, c_2, c_3 = [pow(g, x[i], n) for i in range(4)]

# Bob now does
c = (c_0 * c_1 * c_2 * c_3) % n
c == pow(g,x_sum,n)

True

## DF ZKP for $a < x < b$

DF range proofs build on the positive number proof above. For range [a,b] the following must be true if x lies in that range: $(x-a, b-x)\in \mathbb{N}$

* Actors agree on bases $(g,h)$ and prime $p$.
* Bob sends Alice random challenge $r$
* Every positive value can be [represented with four squares](http://oeis.org/wiki/Sums_of_squares) $x = \{x_0^2 + x_1^2 + x_2^2 + x_3^2 \vert x_i \in \mathbb{I}\}$ 
* Use [Rabin Shallit](https://math.stackexchange.com/questions/483101/rabin-and-shallit-algorithm) or see [source](https://stackoverflow.com/questions/41524508/express-a-given-number-as-a-sum-of-four-squares)
* Alice creates four commitments, one for each $x_i$ as follows
 1. $r_0, r_1, r_2, r_3 \leftarrow \texttt{randInt() for i in range(4)}$
 2. $c_0 = g^{x^2_0}h^{r_0} \mod n$
 3. $c_1 = g^{x^2_1}h^{r_1} \mod n$
 4. $c_2 = g^{x^2_2}h^{r_2} \mod n$
 5. $c_3 = g^{x^2_3}h^{r_3} \mod n$
* Alice commits $r = r_0 + r_1 + r_2 + r_3$
* Bob checks $c = c_0 \times c_1 \times c_2 \times c_3 == g^x h^r \mod n$
* If Alice has a negative number, she cannot represent it with four squares
* Bob cannot find $x$ without discrete log.

Now Alice proves the range by proving that $(x-a,b-x)>0$

1. $c_1 = g^{b-x}h^{-r} \mod n$
2. $c_2 = g^{x-a}h^r \mod n$
3. Proof of commitments
 * $p_1 = \frac{g^b}{c_1} \mod n \equiv g^xh^r \mod n$
 * $p_2 = \frac{c_2}{g^a} \mod n \equiv g^xh^r \mod n$


In [12]:
def ZKPrandom(bitlength=8):
    x = [number.getRandomNBitInteger(8) for i in range(4)]
    return x, int(np.sum(x))

def ZKPnumber(bitlength=4):
    x = [number.getRandomNBitInteger(bitlength) ** 2 for i in range(4)]
    return x, int(np.sum(x))

def ZKPpositive(x, g=3, h=5, n=pow(2,19)-1):
    c_0, c_1, c_2, c_3 = [(pow(g, x[i], n) * pow(h, r[i], n)) % n for i in range(4)]
    c = (c_0 * c_1 * c_2 * c_3) % n
    return c

In [24]:
# Alice and Bob agree on bases g,h and prime n
g, h, n = 3, 5, pow(2,19)-1

x, x_sum = ZKPnumber(4)
r, r_sum = ZKPrandom(8)

print("Alice has number %d and generates random shares" % (x_sum),r)

x_minus_a, x_minus_a_sum = ZKPnumber(2)
b_minus_x, b_minus_x_sum = ZKPnumber(6)

print("Range is [%d,%d]" % (x_sum - x_minus_a_sum, b_minus_x_sum + x_sum))

ZKPpositive(x_minus_a) == pow(g,x_minus_a_sum,n) * pow(h,r_sum,n) % n, ZKPpositive(b_minus_x) == pow(g,b_minus_x_sum,n) * pow(h,r_sum,n) % n

Alice has number 491 and generates random shares [207, 197, 226, 146]
Range is [465,7640]


(True, True)

In [22]:
# I do not know why you would need h here. Probably to provide added security for x but I am not sure.
# To simplify I omit the h part as it is trivially easy to add.

# Alice and Bob agree on bases g,h and prime n
g, h, n = 3, 5, pow(2,19)-1
a, b = 1, 20000

# Bob sends over r to Alice. The implementation flips the process to make it easier.
r = []
r.extend([number.getRandomNBitInteger(8) for i in range(4)]) # Alice finds elements. np.sum(r) is what Bob knows.

# Alice number, normally you would find the elements but this is easier
x = [number.getRandomNBitInteger(4) ** 2 for i in range(4)]
print('Alice number shares are', x, 'which sums to', np.sum(x))

c_0, c_1, c_2, c_3 = [(pow(g, x[i], n) * pow(h, r[i], n)) % n for i in range(4)]

# Bob can verify that the committed number is positive
c = (c_0 * c_1 * c_2 * c_3) % n
assert c == pow(g,int(np.sum(x)),n) * pow(h,int(np.sum(r)),n) % n

# Alice now commits the range proof
c1 = pow(g, int(b - np.sum(x)), n) * pow(g, libnum.invmod(int(np.sum(r)), n), n) % n
c2 = pow(g, int(np.sum(x) - a), n) * pow(h, int(np.sum(r)), n) % n

p1 = pow(g, b, n) * pow(c1, -1, n) % n
p2 = c2 * pow(g, libnum.invmod(a, n), n) % n
p1,p2,pow(g, int(np.sum(x)),n) * pow(h, int(np.sum(r)), n) %n

Alice number shares are [121, 100, 169, 100] which sums to 490


(29237, 210650, 210650)

In [23]:
b - np.sum(x)

19510

In [ ]:
import random
import sys
import libnum

n=pow(2,19)-1

a=5
b=22


x=10

def lipmaa(val):

  for i in range(0,100000):
    for j in range(0,1000):
      for k in range(0,100):
        for l in range(0,10):
          if (((i*i) + (j*j) + (k*k) + (l*l)) == val):     return (i,j,k,l)
  return(0)

def positiveProof(x,r,n):

  if x<0: print("Cannot prove")
          sys.exit(0)
          x0,x1,x2,x3=lipmaa(x)
          r0=random.randint(0,n)
          r1=random.randint(0,n)
          r2=random.randint(0,n)
          r3=random.randint(0,n)
          r0=(r-r1-r2-r3)
          c0=(pow(g,x0*x0,n) * pow(h,r0,n)) % n
          c1=(pow(g,x1*x1,n) * pow(h,r1,n)) % n
          c2=(pow(g,x2*x2,n) * pow(h,r2,n)) % n
          c3=(pow(g,x3*x3,n) * pow(h,r3,n)) % n
          return(c0,c1,c2,c3)
          if (len(sys.argv)>1):
	x=int(sys.argv[1])
if (len(sys.argv)>2):
	a=int(sys.argv[2])
if (len(sys.argv)>3):
	b=int(sys.argv[3])



g=3
h=4

r=random.randint(0,n)

print ("x=",x)
print ("a=",a)
print ("b=",b)
print ("r=",r)

c1_0,c1_1,c1_2,c1_3=positiveProof(b-x,-r,n)
print ("\nb-x=",b-x)
commit1=(c1_0*c1_1*c1_2*c1_3)%n


c2_0,c2_1,c2_2,c2_3=positiveProof(x-a,r,n)
print ("\nx-a=",x-a)
commit2=(c2_0*c2_1*c2_2*c2_3)%n



print ("\nCommit 1: ",c1_0,c1_1,c1_2,c1_3)
print ("Commit 2: ",c2_0,c2_1,c2_2,c2_3)

prover1 =(pow(g,b,n) * pow(commit1,-1,n)) % n

prover2 =(commit2 * pow(g,a,n)) % n


print ("Prover 1: ",prover1)
print ("Prover 2: ",prover2)

prover3 =(pow(g,x,n) * pow(h,r,n)) % n

print ("Prover 3: ",prover3)

if (prover1==prover2):
  print(" Peggy has proven that her value is between: ",a," and ",b)
else: 
  print(" Peggy has NOT proven that her value is between: ",a," and ",b)



a=x+1
b=x+5
prover1 =(pow(g,b,n) * pow(commit1,-1,n)) % n

prover2 =(commit2 * pow(g,a,n)) % n


print ("Prover 1: ",prover1)
print ("Prover 2: ",prover2)

prover3 =(pow(g,x,n) * pow(h,r,n)) % n

print ("Prover 3: ",prover3)

if (prover1==prover2):
  print(" Peggy has proven that her value is between: ",a," and ",b)
else:
  print(" Peggy has NOT proven that her value is between: ",a," and ",b)